In [1]:
# Implementing the art painting style explainable classifier, anyxi.
# Vicent Costa, October, 2021
import numpy as np
import pandas as pd
from statistics import mean
from scipy.stats.mstats import gmean
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Reading data: the training dataset
training_5 = "train5"
training_5 = pd.read_csv(training_5, sep =",",
                        decimal = ".",
                        )
#training_5
#training_5.info() #checking everything is correct.

In [3]:
# Reading data: the testing dataset
test_5="test5"
test_5=pd.read_csv(test_5,sep=",", 
                  decimal='.', 
                  )
#test_5
#test_5.info() #checking everything is correct.

In [4]:
# Definition of the Lukasiewicz connectives.
def LukasiewiczConjunction(conjuncts):
    """
    This function is the semantic interpretation of the Lukasiewicz strong conjunction (for any number of conjuncts). 
    The input is a list of the conjuncts, and the output is the truth value of the conjunction.
    The function warns in case the input is incorrect.
    """
    if len(conjuncts)<2: 
        return print("Warning: conjunction is a binary connective (data is incomplete).")
    negative_conjuncts = list(filter(lambda x: x <0 or x >1, conjuncts))
    if negative_conjuncts !=[]:
        return print("Warning: at least one color trait is incorrect.")
    else:
        return round(max(0,sum(conjuncts)-(len(conjuncts)-1)),14)
#LukasiewiczConjunction([0.1,1,1])    

def LukasiewiczImplication(antecedent,consequent):
    """
    This function is the semantic interpretation of the Lukasiewicz residual implication. 
    The input is the truth value of the antecedent and the truth value of the consequent (as an ordered pair, 
    in the order mentionned), and the output is the truth value of the implication.
    The function warns in case the input is incorrect.
    We do not consider empty antecedents.
    """
    if antecedent is []:
        print("We do not consider empty antecedents.")
    if (antecedent not in pd.Interval(left = 0, right = 1) or consequent not in pd.Interval(left = 0, right = 1)) and antecedent*consequent != 0:
        print("Warning: the antecedent or consequent truth value is incorrect.", antecedent, consequent)
    else:
        return min(1-antecedent+consequent, 1)
#LukasiewiczImplication(0.5,0.45)

In [5]:
#Defining the color traits and spliting the dataset into the three art styles.
colortraits = ["darkness_level",
              "no_paleness_level",
              "contrast_level",
              "red_colors", 
              "diversityofhues",
              "diversityofqcds",
              "bluish_level",
              "greyish_level",
              "vividness",
              "warmth",
              "contrast_yellow_blue", 
              "contrast_red_green"
              ]
baroque = 1.0
baroque_training_5 = training_5[training_5['Classification'] == baroque]
#baroque_training_5
impressionism = 2.0
impressionism_training_5 = training_5[training_5['Classification'] == impressionism]
#impressionism_training_5
postimpressionism = 3.0
postimpressionism_training_5 = training_5[training_5['Classification'] == postimpressionism]
#postimpressionism_training_5

In [6]:
# Training the algorithm: obtining the thresholds
thresholds = []
thresholds.append([baroque_training_5.darkness_level.median(),
                   baroque_training_5.no_paleness_level.median(),
                   baroque_training_5.contrast_level.median(),
                   baroque_training_5.red_colours.median(),
                   impressionism_training_5.diversityofhues.median(),
                   impressionism_training_5.diversityofqcds.median(),
                   impressionism_training_5.bluish_level.median(),
                   impressionism_training_5.greyish_level.median(),
                   postimpressionism_training_5.vividness.median(),
                   postimpressionism_training_5.warmth.median(),
                   postimpressionism_training_5.contrast_yellow_blue.median(),
                   postimpressionism_training_5.contrast_red_green.median()]                   
                 )
#thresholds

In [7]:
# We need to see the paintings of the testing dataset as well as its style.
test_5

,darkness_level,no_paleness_level,contrast_level,red_colours,diversityofhues,diversityofqcds,bluish_level,greyish_level,vividness,warmth,contrast_yellow_blue,contrast_red_green,Classification
0,0.379710,0.753223,0.626487,0.210526,0.727273,0.513514,0.157895,0.386107,0.045177,0.325037,0.209469,0.273663,2.0
1,0.540898,0.750000,0.790898,0.263158,0.727273,0.513514,0.210526,0.613673,0.001804,0.043705,0.214135,0.040096,2.0
2,0.429285,0.707246,0.722039,0.470588,0.636364,0.459459,0.000000,0.315542,0.050475,0.544928,0.011494,0.533533,2.0
3,0.410000,0.850849,0.558142,0.350000,0.636364,0.540541,0.000000,0.219381,0.087313,0.336961,0.426174,0.259590,2.0
4,0.083358,0.655072,0.428286,0.909091,0.727273,0.594595,0.181818,0.580510,0.009795,0.026587,0.204707,0.029985,2.0
5,0.380000,0.807857,0.569364,0.280000,0.909091,0.783784,0.137931,0.279383,0.052558,0.365007,0.262530,0.378456,2.0
6,0.699920,0.874799,0.825121,0.700000,0.363636,0.270270,0.000000,0.172164,0.099759,0.471440,0.000000,0.471440,2.0
7,0.499400,0.895800,0.603600,0.615385,0.454545,0.351351,0.000000,0.207500,0.009700,0.283900,0.000000,0.283900,2.0
8,0.146827,0.539830,0.606997,0.400000,0.727273,0.540541,0.150000,0.743028,0.057271,0.207096,0.150000,0.207096,2.0
9,0.313134,0.535223,0.777912,0.500000,0.454545,0.378378,0.000000,0.097980,0.181087,0.901718,0.138881,0.762838,2.0


In [8]:
# Obtaining the truth value of each evaluated atomic formula related 
#to each color trait. Training 0 dataset. 
test_5_evaluated = test_5
test_5_evaluated['darkness_level'][0] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][0])
test_5_evaluated['darkness_level'][1] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][1])
test_5_evaluated['darkness_level'][2] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][2])
test_5_evaluated['darkness_level'][3] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][3])
test_5_evaluated['darkness_level'][4] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][4])
test_5_evaluated['darkness_level'][5] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][5])
test_5_evaluated['darkness_level'][6] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][6])
test_5_evaluated['darkness_level'][7] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][7])
test_5_evaluated['darkness_level'][8] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][8])
test_5_evaluated['darkness_level'][9] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][9])
test_5_evaluated['darkness_level'][10] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][10])
test_5_evaluated['darkness_level'][11] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][11])
test_5_evaluated['darkness_level'][12] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][12])
test_5_evaluated['darkness_level'][13] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][13])
test_5_evaluated['darkness_level'][14] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][14])
test_5_evaluated['darkness_level'][15] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][15])
test_5_evaluated['darkness_level'][16] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][16])
test_5_evaluated['darkness_level'][17] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][17])
test_5_evaluated['darkness_level'][18] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][18])
test_5_evaluated['darkness_level'][19] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][19])
test_5_evaluated['darkness_level'][20] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][20])
test_5_evaluated['darkness_level'][21] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][21])
test_5_evaluated['darkness_level'][22] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][22])
test_5_evaluated['darkness_level'][23] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][23])
test_5_evaluated['darkness_level'][24] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][24])
test_5_evaluated['darkness_level'][25] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][25])
test_5_evaluated['darkness_level'][26] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][26])
test_5_evaluated['darkness_level'][27] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][27])
test_5_evaluated['darkness_level'][28] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][28])
test_5_evaluated['darkness_level'][29] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][29])
test_5_evaluated['darkness_level'][30] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][30])
test_5_evaluated['darkness_level'][31] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][31])
test_5_evaluated['darkness_level'][32] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][32])
test_5_evaluated['darkness_level'][33] = LukasiewiczImplication(thresholds[0][0],test_5_evaluated['darkness_level'][33])
test_5_evaluated['no_paleness_level'][0] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][0])
test_5_evaluated['no_paleness_level'][1] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][1])
test_5_evaluated['no_paleness_level'][2] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][2])
test_5_evaluated['no_paleness_level'][3] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][3])
test_5_evaluated['no_paleness_level'][4] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][4])
test_5_evaluated['no_paleness_level'][5] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][5])
test_5_evaluated['no_paleness_level'][6] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][6])
test_5_evaluated['no_paleness_level'][7] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][7])
test_5_evaluated['no_paleness_level'][8] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][8])
test_5_evaluated['no_paleness_level'][9] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][9])
test_5_evaluated['no_paleness_level'][10] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][10])
test_5_evaluated['no_paleness_level'][11] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][11])
test_5_evaluated['no_paleness_level'][12] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][12])
test_5_evaluated['no_paleness_level'][13] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][13])
test_5_evaluated['no_paleness_level'][14] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][14])
test_5_evaluated['no_paleness_level'][15] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][15])
test_5_evaluated['no_paleness_level'][16] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][16])
test_5_evaluated['no_paleness_level'][17] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][17])
test_5_evaluated['no_paleness_level'][18] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][18])
test_5_evaluated['no_paleness_level'][19] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][19])
test_5_evaluated['no_paleness_level'][20] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][20])
test_5_evaluated['no_paleness_level'][21] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][21])
test_5_evaluated['no_paleness_level'][22] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][22])
test_5_evaluated['no_paleness_level'][23] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][23])
test_5_evaluated['no_paleness_level'][24] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][24])
test_5_evaluated['no_paleness_level'][25] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][25])
test_5_evaluated['no_paleness_level'][26] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][26])
test_5_evaluated['no_paleness_level'][27] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][27])
test_5_evaluated['no_paleness_level'][28] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][28])
test_5_evaluated['no_paleness_level'][29] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][29])
test_5_evaluated['no_paleness_level'][30] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][30])
test_5_evaluated['no_paleness_level'][31] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][31])
test_5_evaluated['no_paleness_level'][32] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][32])
test_5_evaluated['no_paleness_level'][33] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['no_paleness_level'][33])
test_5_evaluated['contrast_level'][0] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][0])
test_5_evaluated['contrast_level'][1] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][1])
test_5_evaluated['contrast_level'][2] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][2])
test_5_evaluated['contrast_level'][3] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][3])
test_5_evaluated['contrast_level'][4] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][4])
test_5_evaluated['contrast_level'][5] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][5])
test_5_evaluated['contrast_level'][6] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][6])
test_5_evaluated['contrast_level'][7] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][7])
test_5_evaluated['contrast_level'][8] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][8])
test_5_evaluated['contrast_level'][9] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][9])
test_5_evaluated['contrast_level'][10] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][10])
test_5_evaluated['contrast_level'][11] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][11])
test_5_evaluated['contrast_level'][12] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][12])
test_5_evaluated['contrast_level'][13] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][13])
test_5_evaluated['contrast_level'][14] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][14])
test_5_evaluated['contrast_level'][15] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][15])
test_5_evaluated['contrast_level'][16] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][16])
test_5_evaluated['contrast_level'][17] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][17])
test_5_evaluated['contrast_level'][18] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][18])
test_5_evaluated['contrast_level'][19] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][19])
test_5_evaluated['contrast_level'][20] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][20])
test_5_evaluated['contrast_level'][21] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][21])
test_5_evaluated['contrast_level'][22] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][22])
test_5_evaluated['contrast_level'][23] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][23])
test_5_evaluated['contrast_level'][24] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][24])
test_5_evaluated['contrast_level'][25] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][25])
test_5_evaluated['contrast_level'][26] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][26])
test_5_evaluated['contrast_level'][27] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][27])
test_5_evaluated['contrast_level'][28] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][28])
test_5_evaluated['contrast_level'][29] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][29])
test_5_evaluated['contrast_level'][30] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][30])
test_5_evaluated['contrast_level'][31] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][31])
test_5_evaluated['contrast_level'][32] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][32])
test_5_evaluated['contrast_level'][33] = LukasiewiczImplication(thresholds[0][2],test_5_evaluated['contrast_level'][33])
test_5_evaluated['red_colours'][0] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][0])
test_5_evaluated['red_colours'][1] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][1])
test_5_evaluated['red_colours'][2] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][2])
test_5_evaluated['red_colours'][3] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][3])
test_5_evaluated['red_colours'][4] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][4])
test_5_evaluated['red_colours'][5] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][5])
test_5_evaluated['red_colours'][6] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][6])
test_5_evaluated['red_colours'][7] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][7])
test_5_evaluated['red_colours'][8] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][8])
test_5_evaluated['red_colours'][9] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][9])
test_5_evaluated['red_colours'][10] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][10])
test_5_evaluated['red_colours'][11] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][11])
test_5_evaluated['red_colours'][12] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][12])
test_5_evaluated['red_colours'][13] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][13])
test_5_evaluated['red_colours'][14] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][14])
test_5_evaluated['red_colours'][15] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][15])
test_5_evaluated['red_colours'][16] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][16])
test_5_evaluated['red_colours'][17] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][17])
test_5_evaluated['red_colours'][18] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][18])
test_5_evaluated['red_colours'][19] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][19])
test_5_evaluated['red_colours'][20] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][20])
test_5_evaluated['red_colours'][21] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][21])
test_5_evaluated['red_colours'][22] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][22])
test_5_evaluated['red_colours'][23] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][23])
test_5_evaluated['red_colours'][24] = LukasiewiczImplication(thresholds[0][3],test_5_evaluated['red_colours'][24])
test_5_evaluated['red_colours'][25] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['red_colours'][25])
test_5_evaluated['red_colours'][26] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['red_colours'][26])
test_5_evaluated['red_colours'][27] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['red_colours'][27])
test_5_evaluated['red_colours'][28] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['red_colours'][28])
test_5_evaluated['red_colours'][29] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['red_colours'][29])
test_5_evaluated['red_colours'][30] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['red_colours'][30])
test_5_evaluated['red_colours'][31] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['red_colours'][31])
test_5_evaluated['red_colours'][32] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['red_colours'][32])
test_5_evaluated['red_colours'][33] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['red_colours'][33])
test_5_evaluated['diversityofhues'][0] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][0])
test_5_evaluated['diversityofhues'][1] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][1])
test_5_evaluated['diversityofhues'][2] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][2])
test_5_evaluated['diversityofhues'][3] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][3])
test_5_evaluated['diversityofhues'][4] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][4])
test_5_evaluated['diversityofhues'][5] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][5])
test_5_evaluated['diversityofhues'][6] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][6])
test_5_evaluated['diversityofhues'][7] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][7])
test_5_evaluated['diversityofhues'][8] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][8])
test_5_evaluated['diversityofhues'][9] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][9])
test_5_evaluated['diversityofhues'][10] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][10])
test_5_evaluated['diversityofhues'][11] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][11])
test_5_evaluated['diversityofhues'][12] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][12])
test_5_evaluated['diversityofhues'][13] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][13])
test_5_evaluated['diversityofhues'][14] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][14])
test_5_evaluated['diversityofhues'][15] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][15])
test_5_evaluated['diversityofhues'][16] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][16])
test_5_evaluated['diversityofhues'][17] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][17])
test_5_evaluated['diversityofhues'][18] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][18])
test_5_evaluated['diversityofhues'][19] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][19])
test_5_evaluated['diversityofhues'][20] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][20])
test_5_evaluated['diversityofhues'][21] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][21])
test_5_evaluated['diversityofhues'][22] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][22])
test_5_evaluated['diversityofhues'][23] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][23])
test_5_evaluated['diversityofhues'][24] = LukasiewiczImplication(thresholds[0][4],test_5_evaluated['diversityofhues'][24])
test_5_evaluated['diversityofhues'][25] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['diversityofhues'][25])
test_5_evaluated['diversityofhues'][26] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['diversityofhues'][26])
test_5_evaluated['diversityofhues'][27] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['diversityofhues'][27])
test_5_evaluated['diversityofhues'][28] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['diversityofhues'][28])
test_5_evaluated['diversityofhues'][29] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['diversityofhues'][29])
test_5_evaluated['diversityofhues'][30] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['diversityofhues'][30])
test_5_evaluated['diversityofhues'][31] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['diversityofhues'][31])
test_5_evaluated['diversityofhues'][32] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['diversityofhues'][32])
test_5_evaluated['diversityofhues'][33] = LukasiewiczImplication(thresholds[0][1],test_5_evaluated['diversityofhues'][33])
test_5_evaluated['diversityofqcds'][0] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][0])
test_5_evaluated['diversityofqcds'][1] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][1])
test_5_evaluated['diversityofqcds'][2] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][2])
test_5_evaluated['diversityofqcds'][3] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][3])
test_5_evaluated['diversityofqcds'][4] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][4])
test_5_evaluated['diversityofqcds'][5] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][5])
test_5_evaluated['diversityofqcds'][6] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][6])
test_5_evaluated['diversityofqcds'][7] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][7])
test_5_evaluated['diversityofqcds'][8] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][8])
test_5_evaluated['diversityofqcds'][9] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][9])
test_5_evaluated['diversityofqcds'][10] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][10])
test_5_evaluated['diversityofqcds'][11] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][11])
test_5_evaluated['diversityofqcds'][12] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][12])
test_5_evaluated['diversityofqcds'][13] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][13])
test_5_evaluated['diversityofqcds'][14] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][14])
test_5_evaluated['diversityofqcds'][15] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][15])
test_5_evaluated['diversityofqcds'][16] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][16])
test_5_evaluated['diversityofqcds'][17] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][17])
test_5_evaluated['diversityofqcds'][18] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][18])
test_5_evaluated['diversityofqcds'][19] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][19])
test_5_evaluated['diversityofqcds'][20] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][20])
test_5_evaluated['diversityofqcds'][21] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][21])
test_5_evaluated['diversityofqcds'][22] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][22])
test_5_evaluated['diversityofqcds'][23] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][23])
test_5_evaluated['diversityofqcds'][24] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][24])
test_5_evaluated['diversityofqcds'][25] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][25])
test_5_evaluated['diversityofqcds'][26] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][26])
test_5_evaluated['diversityofqcds'][27] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][27])
test_5_evaluated['diversityofqcds'][28] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][28])
test_5_evaluated['diversityofqcds'][29] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][29])
test_5_evaluated['diversityofqcds'][30] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][30])
test_5_evaluated['diversityofqcds'][31] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][31])
test_5_evaluated['diversityofqcds'][32] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][32])
test_5_evaluated['diversityofqcds'][33] = LukasiewiczImplication(thresholds[0][5],test_5_evaluated['diversityofqcds'][33])
test_5_evaluated['bluish_level'][0] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][0])
test_5_evaluated['bluish_level'][1] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][1])
test_5_evaluated['bluish_level'][2] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][2])
test_5_evaluated['bluish_level'][3] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][3])
test_5_evaluated['bluish_level'][4] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][4])
test_5_evaluated['bluish_level'][5] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][5])
test_5_evaluated['bluish_level'][6] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][6])
test_5_evaluated['bluish_level'][7] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][7])
test_5_evaluated['bluish_level'][8] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][8])
test_5_evaluated['bluish_level'][9] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][9])
test_5_evaluated['bluish_level'][10] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][10])
test_5_evaluated['bluish_level'][11] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][11])
test_5_evaluated['bluish_level'][12] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][12])
test_5_evaluated['bluish_level'][13] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][13])
test_5_evaluated['bluish_level'][14] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][14])
test_5_evaluated['bluish_level'][15] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][15])
test_5_evaluated['bluish_level'][16] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][16])
test_5_evaluated['bluish_level'][17] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][17])
test_5_evaluated['bluish_level'][18] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][18])
test_5_evaluated['bluish_level'][19] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][19])
test_5_evaluated['bluish_level'][20] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][20])
test_5_evaluated['bluish_level'][21] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][21])
test_5_evaluated['bluish_level'][22] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][22])
test_5_evaluated['bluish_level'][23] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][23])
test_5_evaluated['bluish_level'][24] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][24])
test_5_evaluated['bluish_level'][25] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][25])
test_5_evaluated['bluish_level'][26] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][26])
test_5_evaluated['bluish_level'][27] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][27])
test_5_evaluated['bluish_level'][28] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][28])
test_5_evaluated['bluish_level'][29] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][29])
test_5_evaluated['bluish_level'][30] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][30])
test_5_evaluated['bluish_level'][31] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][31])
test_5_evaluated['bluish_level'][32] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][32])
test_5_evaluated['bluish_level'][33] = LukasiewiczImplication(thresholds[0][6],test_5_evaluated['bluish_level'][33])
test_5_evaluated['greyish_level'][0] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][0])
test_5_evaluated['greyish_level'][1] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][1])
test_5_evaluated['greyish_level'][2] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][2])
test_5_evaluated['greyish_level'][3] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][3])
test_5_evaluated['greyish_level'][4] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][4])
test_5_evaluated['greyish_level'][5] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][5])
test_5_evaluated['greyish_level'][6] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][6])
test_5_evaluated['greyish_level'][7] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][7])
test_5_evaluated['greyish_level'][8] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][8])
test_5_evaluated['greyish_level'][9] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][9])
test_5_evaluated['greyish_level'][10] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][10])
test_5_evaluated['greyish_level'][11] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][11])
test_5_evaluated['greyish_level'][12] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][12])
test_5_evaluated['greyish_level'][13] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][13])
test_5_evaluated['greyish_level'][14] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][14])
test_5_evaluated['greyish_level'][15] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][15])
test_5_evaluated['greyish_level'][16] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][16])
test_5_evaluated['greyish_level'][17] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][17])
test_5_evaluated['greyish_level'][18] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][18])
test_5_evaluated['greyish_level'][19] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][19])
test_5_evaluated['greyish_level'][20] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][20])
test_5_evaluated['greyish_level'][21] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][21])
test_5_evaluated['greyish_level'][22] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][22])
test_5_evaluated['greyish_level'][23] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][23])
test_5_evaluated['greyish_level'][24] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][24])
test_5_evaluated['greyish_level'][25] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][25])
test_5_evaluated['greyish_level'][26] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][26])
test_5_evaluated['greyish_level'][27] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][27])
test_5_evaluated['greyish_level'][28] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][28])
test_5_evaluated['greyish_level'][29] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][29])
test_5_evaluated['greyish_level'][30] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][30])
test_5_evaluated['greyish_level'][31] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][31])
test_5_evaluated['greyish_level'][32] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][32])
test_5_evaluated['greyish_level'][33] = LukasiewiczImplication(thresholds[0][7],test_5_evaluated['greyish_level'][33])
test_5_evaluated['vividness'][0] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][0])
test_5_evaluated['vividness'][1] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][1])
test_5_evaluated['vividness'][2] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][2])
test_5_evaluated['vividness'][3] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][3])
test_5_evaluated['vividness'][4] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][4])
test_5_evaluated['vividness'][5] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][5])
test_5_evaluated['vividness'][6] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][6])
test_5_evaluated['vividness'][7] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][7])
test_5_evaluated['vividness'][8] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][8])
test_5_evaluated['vividness'][9] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][9])
test_5_evaluated['vividness'][10] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][10])
test_5_evaluated['vividness'][11] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][11])
test_5_evaluated['vividness'][12] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][12])
test_5_evaluated['vividness'][13] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][13])
test_5_evaluated['vividness'][14] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][14])
test_5_evaluated['vividness'][15] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][15])
test_5_evaluated['vividness'][16] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][16])
test_5_evaluated['vividness'][17] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][17])
test_5_evaluated['vividness'][18] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][18])
test_5_evaluated['vividness'][19] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][19])
test_5_evaluated['vividness'][20] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][20])
test_5_evaluated['vividness'][21] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][21])
test_5_evaluated['vividness'][22] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][22])
test_5_evaluated['vividness'][23] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][23])
test_5_evaluated['vividness'][24] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][24])
test_5_evaluated['vividness'][25] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][25])
test_5_evaluated['vividness'][26] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][26])
test_5_evaluated['vividness'][27] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][27])
test_5_evaluated['vividness'][28] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][28])
test_5_evaluated['vividness'][29] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][29])
test_5_evaluated['vividness'][30] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][30])
test_5_evaluated['vividness'][31] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][31])
test_5_evaluated['vividness'][32] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][32])
test_5_evaluated['vividness'][33] = LukasiewiczImplication(thresholds[0][8],test_5_evaluated['vividness'][33])
test_5_evaluated['warmth'][0] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][0])
test_5_evaluated['warmth'][1] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][1])
test_5_evaluated['warmth'][2] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][2])
test_5_evaluated['warmth'][3] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][3])
test_5_evaluated['warmth'][4] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][4])
test_5_evaluated['warmth'][5] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][5])
test_5_evaluated['warmth'][6] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][6])
test_5_evaluated['warmth'][7] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][7])
test_5_evaluated['warmth'][8] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][8])
test_5_evaluated['warmth'][9] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][9])
test_5_evaluated['warmth'][10] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][10])
test_5_evaluated['warmth'][11] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][11])
test_5_evaluated['warmth'][12] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][12])
test_5_evaluated['warmth'][13] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][13])
test_5_evaluated['warmth'][14] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][14])
test_5_evaluated['warmth'][15] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][15])
test_5_evaluated['warmth'][16] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][16])
test_5_evaluated['warmth'][17] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][17])
test_5_evaluated['warmth'][18] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][18])
test_5_evaluated['warmth'][19] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][19])
test_5_evaluated['warmth'][20] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][20])
test_5_evaluated['warmth'][21] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][21])
test_5_evaluated['warmth'][22] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][22])
test_5_evaluated['warmth'][23] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][23])
test_5_evaluated['warmth'][24] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][24])
test_5_evaluated['warmth'][25] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][25])
test_5_evaluated['warmth'][26] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][26])
test_5_evaluated['warmth'][27] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][27])
test_5_evaluated['warmth'][28] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][28])
test_5_evaluated['warmth'][29] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][29])
test_5_evaluated['warmth'][30] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][30])
test_5_evaluated['warmth'][31] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][31])
test_5_evaluated['warmth'][32] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][32])
test_5_evaluated['warmth'][33] = LukasiewiczImplication(thresholds[0][9],test_5_evaluated['warmth'][33])
test_5_evaluated['contrast_yellow_blue'][0] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][0])
test_5_evaluated['contrast_yellow_blue'][1] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][1])
test_5_evaluated['contrast_yellow_blue'][2] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][2])
test_5_evaluated['contrast_yellow_blue'][3] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][3])
test_5_evaluated['contrast_yellow_blue'][4] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][4])
test_5_evaluated['contrast_yellow_blue'][5] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][5])
test_5_evaluated['contrast_yellow_blue'][6] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][6])
test_5_evaluated['contrast_yellow_blue'][7] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][7])
test_5_evaluated['contrast_yellow_blue'][8] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][8])
test_5_evaluated['contrast_yellow_blue'][9] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][9])
test_5_evaluated['contrast_yellow_blue'][10] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][10])
test_5_evaluated['contrast_yellow_blue'][11] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][11])
test_5_evaluated['contrast_yellow_blue'][12] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][12])
test_5_evaluated['contrast_yellow_blue'][13] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][13])
test_5_evaluated['contrast_yellow_blue'][14] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][14])
test_5_evaluated['contrast_yellow_blue'][15] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][15])
test_5_evaluated['contrast_yellow_blue'][16] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][16])
test_5_evaluated['contrast_yellow_blue'][17] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][17])
test_5_evaluated['contrast_yellow_blue'][18] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][18])
test_5_evaluated['contrast_yellow_blue'][19] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][19])
test_5_evaluated['contrast_yellow_blue'][20] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][20])
test_5_evaluated['contrast_yellow_blue'][21] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][21])
test_5_evaluated['contrast_yellow_blue'][22] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][22])
test_5_evaluated['contrast_yellow_blue'][23] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][23])
test_5_evaluated['contrast_yellow_blue'][24] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][24])
test_5_evaluated['contrast_yellow_blue'][25] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][25])
test_5_evaluated['contrast_yellow_blue'][26] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][26])
test_5_evaluated['contrast_yellow_blue'][27] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][27])
test_5_evaluated['contrast_yellow_blue'][28] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][28])
test_5_evaluated['contrast_yellow_blue'][29] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][29])
test_5_evaluated['contrast_yellow_blue'][30] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][30])
test_5_evaluated['contrast_yellow_blue'][31] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][31])
test_5_evaluated['contrast_yellow_blue'][32] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][32])
test_5_evaluated['contrast_yellow_blue'][33] = LukasiewiczImplication(thresholds[0][10],test_5_evaluated['contrast_yellow_blue'][33])
test_5_evaluated['contrast_red_green'][0] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][0])
test_5_evaluated['contrast_red_green'][1] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][1])
test_5_evaluated['contrast_red_green'][2] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][2])
test_5_evaluated['contrast_red_green'][3] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][3])
test_5_evaluated['contrast_red_green'][4] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][4])
test_5_evaluated['contrast_red_green'][5] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][5])
test_5_evaluated['contrast_red_green'][6] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][6])
test_5_evaluated['contrast_red_green'][7] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][7])
test_5_evaluated['contrast_red_green'][8] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][8])
test_5_evaluated['contrast_red_green'][9] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][9])
test_5_evaluated['contrast_red_green'][10] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][10])
test_5_evaluated['contrast_red_green'][11] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][11])
test_5_evaluated['contrast_red_green'][12] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][12])
test_5_evaluated['contrast_red_green'][13] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][13])
test_5_evaluated['contrast_red_green'][14] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][14])
test_5_evaluated['contrast_red_green'][15] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][15])
test_5_evaluated['contrast_red_green'][16] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][16])
test_5_evaluated['contrast_red_green'][17] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][17])
test_5_evaluated['contrast_red_green'][18] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][18])
test_5_evaluated['contrast_red_green'][19] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][19])
test_5_evaluated['contrast_red_green'][20] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][20])
test_5_evaluated['contrast_red_green'][21] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][21])
test_5_evaluated['contrast_red_green'][22] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][22])
test_5_evaluated['contrast_red_green'][23] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][23])
test_5_evaluated['contrast_red_green'][24] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][24])
test_5_evaluated['contrast_red_green'][25] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][25])
test_5_evaluated['contrast_red_green'][26] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][26])
test_5_evaluated['contrast_red_green'][27] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][27])
test_5_evaluated['contrast_red_green'][28] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][28])
test_5_evaluated['contrast_red_green'][29] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][29])
test_5_evaluated['contrast_red_green'][30] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][30])
test_5_evaluated['contrast_red_green'][31] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][31])
test_5_evaluated['contrast_red_green'][32] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][32])
test_5_evaluated['contrast_red_green'][33] = LukasiewiczImplication(thresholds[0][11],test_5_evaluated['contrast_red_green'][33])
test_5

,darkness_level,no_paleness_level,contrast_level,red_colours,diversityofhues,diversityofqcds,bluish_level,greyish_level,vividness,warmth,contrast_yellow_blue,contrast_red_green,Classification
0,0.549710,0.844991,0.691013,0.743860,1.000000,1.000000,1.000000,0.940049,0.965625,0.828328,1.000000,0.847811,2.0
1,0.710898,0.841768,0.855424,0.796491,1.000000,1.000000,1.000000,1.000000,0.922252,0.546995,1.000000,0.614244,2.0
2,0.599285,0.799014,0.786565,1.000000,1.000000,1.000000,0.900000,0.869485,0.970922,1.000000,0.837247,1.000000,2.0
3,0.580000,0.942617,0.622668,0.883333,1.000000,1.000000,0.900000,0.773324,1.000000,0.840251,1.000000,0.833738,2.0
4,0.253358,0.746840,0.492812,1.000000,1.000000,1.000000,1.000000,1.000000,0.930243,0.529877,1.000000,0.604133,2.0
5,0.550000,0.899625,0.633890,0.813333,1.000000,1.000000,1.000000,0.833326,0.973006,0.868297,1.000000,0.952604,2.0
6,0.869920,0.966567,0.889646,1.000000,0.727273,0.810811,0.900000,0.726107,1.000000,0.974730,0.825753,1.000000,2.0
7,0.669400,0.987568,0.668126,1.000000,0.818182,0.891892,0.900000,0.761443,0.930148,0.787190,0.825753,0.858048,2.0
8,0.316827,0.631598,0.671522,0.933333,1.000000,1.000000,1.000000,1.000000,0.977719,0.710387,0.975753,0.781244,2.0
9,0.483134,0.626990,0.842438,1.000000,0.818182,0.918919,0.900000,0.651923,1.000000,1.000000,0.964633,1.000000,2.0


In [9]:
# Obtaining the membership degrees of each art style (Costa, Dellunde, Falomir, 2021).
baroquetraitsevaluated= [
                          "darkness_level",
                          "no_paleness_level",
                          "contrast_level",
                          "red_colours", 
                        ]

for i in range(len(test_5)):
    test_5['baroque_level'] = test_5[baroquetraitsevaluated].apply(lambda row: LukasiewiczConjunction(list(row)),axis=1)

impressionismtraitsevaluated = [ "diversityofhues",
              "diversityofqcds",
              "bluish_level",
              "greyish_level"]    

for i in range(len(test_5)):
    test_5['impressionism_level'] = test_5[impressionismtraitsevaluated].apply(lambda row: LukasiewiczConjunction(list(row)),axis=1)
    
postimpressionismtraitsevaluated = ["vividness",
              "warmth",
              "contrast_yellow_blue",
              "contrast_red_green",
                                   ]

for i in range(len(test_5)):
    test_5['postimpressionism_level'] = test_5[postimpressionismtraitsevaluated].apply(lambda row: LukasiewiczConjunction(list(row)),axis=1)

#test_5

In [10]:
# Definition of the explanation functions
# First, we explicit the text of the explanations.
explanations = [
                        "The darkness evidences the Baroque style.",
                        "The contrast of dark and pale colors evidences the Baroque style.",
                        "The lack of pale colors evidences the Baroque style.",
                        "The level of reddish evidences the Baroque style.",
                        "The diversity of qualitative colors evidences the Impressionism style.",
                        "The variety of hues evidences the Impressionism style.",
                        "The amount of bluish evidences the Impressionism style.",
                        "The amount of grey evidences the Impressionism style.",
                        "The presence of vivid colors evidences the Post-Impressionism style.",
                        "The high level of warm colors evidences the Post-Impressionism style.",
                        "The contrast between blue and yellow evidences the Post-Impressionism style.",
                        "The contrast between red and green evidences the Post-Impressionism style.",
                        ]

In [11]:
#Second, we select the Baroque paintings from the testing dataset, the Impressionist ones,
# and the Post-Impressionist painitngs. 
baroque = 1.0
baroque_test_5 = test_5[test_5['Classification'] == baroque]
#baroque_test_5
impressionism = 2.0
impressionism_test_5 = test_5[test_5['Classification'] == impressionism]
#impressionism_test_5
postimpressionism = 3.0
postimpressionism_test_5 = test_5[test_5['Classification'] == postimpressionism]

In [12]:
# Then, we obtain the thresholds for the explanation functions.
# We do it for the mean, the median and the geometric mean. 

thresholds_explanations = []
thresholds_explanations.append([baroque_test_5.darkness_level.mean(),
                   baroque_test_5.no_paleness_level.mean(),
                   baroque_test_5.contrast_level.mean(),
                   baroque_test_5.red_colours.mean(),
                   impressionism_test_5.diversityofhues.mean(),
                   impressionism_test_5.diversityofqcds.mean(),
                   impressionism_test_5.bluish_level.mean(),
                   impressionism_test_5.greyish_level.mean(),
                   postimpressionism_test_5.vividness.mean(),
                   postimpressionism_test_5.warmth.mean(),
                   postimpressionism_test_5.contrast_yellow_blue.mean(),
                   postimpressionism_test_5.contrast_red_green.mean()]                   
                 )
#thresholds_explanations[0][10]

In [13]:
thresholds_explanations_median = []
thresholds_explanations_median.append([baroque_test_5.darkness_level.median(),
                   baroque_test_5.no_paleness_level.median(),
                   baroque_test_5.contrast_level.median(),
                   baroque_test_5.red_colours.median(),
                   impressionism_test_5.diversityofhues.median(),
                   impressionism_test_5.diversityofqcds.median(),
                   impressionism_test_5.bluish_level.median(),
                   impressionism_test_5.greyish_level.median(),
                   postimpressionism_test_5.vividness.median(),
                   postimpressionism_test_5.warmth.median(),
                   postimpressionism_test_5.contrast_yellow_blue.median(),
                   postimpressionism_test_5.contrast_red_green.median()]                   
                 )
#thresholds_explanations_median

In [14]:
thresholds_explanations_geometric_mean = []
thresholds_explanations_geometric_mean.append([gmean(baroque_test_5.darkness_level),
                   gmean(baroque_test_5.no_paleness_level),
                   gmean(baroque_test_5.contrast_level),
                   gmean(baroque_test_5.red_colours),
                   gmean(impressionism_test_5.diversityofhues),
                   gmean(impressionism_test_5.diversityofqcds),
                   gmean(impressionism_test_5.bluish_level),
                   gmean(impressionism_test_5.greyish_level),
                   gmean(postimpressionism_test_5.vividness),
                   gmean(postimpressionism_test_5.warmth),
                   gmean(postimpressionism_test_5.contrast_yellow_blue),
                   gmean(postimpressionism_test_5.contrast_red_green)]                   
                 )
#thresholds_explanations_geometric_mean

In [15]:
#test_5

In [16]:
# Now we indicate which explanations would be activated
# if the Baroque style is selected (testing dataset).

def baroque_explanations(id_painting):
    """
    This function shows the number of the "activated" explanations associated to the Baroque style.
   """
    if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
        a = 1
    else: 
        if (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
            a = 1
        else: 
            if (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][3]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                a = 1
            else:
                if (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][3]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all(): 
                    a = 1
                else:
                    if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                        a = 2
                    else:
                        if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][3]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                            a = 2
                        else:
                            if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][1]).all(): 
                                a = 2
                            else:
                                if (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                                    a = 2
                                else:
                                    if (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all(): 
                                        a = 2
                                    else:
                                        if (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][1]).all(): 
                                            a = 2
                                        else:
                                            if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                                                a = 3
                                            else:
                                                if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all(): 
                                                        a = 3
                                                else:
                                                    if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][1]).all(): 
                                                        a = 3
                                                    else:
                                                        if (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all(): 
                                                            a = 3
                                                        else:
                                                            if (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all(): 
                                                                a = 3
                                                            else:
                                                                a = 0
    return print("The number of Baroque explanations activated is", a)



baroque_explanations(15)

The number of Baroque explanations activated is 3


In [17]:
# Now we indicate which explanations would be activated
# if the Impressionism style is selected (testing dataset).

def impressionism_explanations(id_painting):
    """
    This function shows the number of the "activated" explanations associated to the Impressionism style.
   """
    if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
        a = 1
    else: 
        if (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
            a = 1
        else: 
            if (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][7]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                a = 1
            else:
                if (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][7]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all(): 
                    a = 1
                else:
                    if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                        a = 2
                    else:
                        if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][7]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                            a = 2
                        else:
                            if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][5]).all(): 
                                a = 2
                            else:
                                if (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                                    a = 2
                                else:
                                    if (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all(): 
                                        a = 2
                                    else:
                                        if (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][5]).all(): 
                                            a = 2
                                        else:
                                            if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                                                a = 3
                                            else:
                                                if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all(): 
                                                        a = 3
                                                else:
                                                    if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][5]).all(): 
                                                        a = 3
                                                    else:
                                                        if (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all(): 
                                                            a = 3
                                                        else:
                                                            if (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all(): 
                                                                a = 3
                                                            else:
                                                                a = 0
    return print("The number of activated explanations concerning the Impressionism style is", a)



impressionism_explanations(3)

The number of activated explanations concerning the Impressionism style is 2


In [18]:
# Now we indicate which explanations would be activated
# if the Post-Impressionism style is selected (testing dataset).

def post_impressionism_explanations(id_painting):
    """
    This function shows the number of the "activated" explanations associated to the post_impressionism style.
   """
    if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
        a = 1
    else: 
        if (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
            a = 1
        else: 
            if (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][11]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                a = 1
            else:
                if (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][11]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all(): 
                    a = 1
                else:
                    if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                        a = 2
                    else:
                        if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][11]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                            a = 2
                        else:
                            if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][9]).all(): 
                                a = 2
                            else:
                                if (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                                    a = 2
                                else:
                                    if (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all(): 
                                        a = 2
                                    else:
                                        if (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][9]).all(): 
                                            a = 2
                                        else:
                                            if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                                                a = 3
                                            else:
                                                if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all(): 
                                                        a = 3
                                                else:
                                                    if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][9]).all(): 
                                                        a = 3
                                                    else:
                                                        if (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all(): 
                                                            a = 3
                                                        else:
                                                            if (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all(): 
                                                                a = 3
                                                            else:
                                                                a = 0
    return print("The number of activated explanations concerning the Post-Impressionism style is", a)



post_impressionism_explanations(13)

The number of activated explanations concerning the Post-Impressionism style is 0


In [19]:
#Number of explanations (Baroque)

# Now we indicate which explanations would be activated
# if the Baroque style is selected (testing dataset).

def number_baroque_explanations(id_painting):
    """
    This function obtains the number of the "activated" explanations associated to the Baroque style.
   """
    if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
        a = 1
    else: 
        if (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
            a = 1
        else: 
            if (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][3]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                a = 1
            else:
                if (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][3]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all(): 
                    a = 1
                else:
                    if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                        a = 2
                    else:
                        if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][3]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                            a = 2
                        else:
                            if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][1]).all(): 
                                a = 2
                            else:
                                if (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                                    a = 2
                                else:
                                    if (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all(): 
                                        a = 2
                                    else:
                                        if (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][1]).all(): 
                                            a = 2
                                        else:
                                            if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] < thresholds_explanations[0][3]).all(): 
                                                a = 3
                                            else:
                                                if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['contrast_level'][id_painting] < thresholds_explanations[0][2]).all(): 
                                                        a = 3
                                                else:
                                                    if (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['no_paleness_level'][id_painting] < thresholds_explanations[0][1]).all(): 
                                                        a = 3
                                                    else:
                                                        if (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['darkness_level'][id_painting] < thresholds_explanations[0][0]).all(): 
                                                            a = 3
                                                        else:
                                                            if (test_5['no_paleness_level'][id_painting] >= thresholds_explanations[0][1]).all() and (test_5['contrast_level'][id_painting] >= thresholds_explanations[0][2]).all() and (test_5['red_colours'][id_painting] >= thresholds_explanations[0][3]).all() and (test_5['darkness_level'][id_painting] >= thresholds_explanations[0][0]).all(): 
                                                                a = 4
                                                            else:
                                                                a = 0
    return a



number_baroque_explanations(15)

4

In [20]:
# Warmth (Baroque)
def baroque_warmth(id_painting):
    """
    This function says if the level of warmth is smaller than 0.32 or if red colors are higher than 0.4 provided that the level of warmth is >= 0.32.
   """
    if (test_5['warmth'][id_painting] < 0.32).all() or (test_5['warmth'][id_painting] >= 0.32 and test_5['red_colours'][id_painting] > 0.4).all():
        b = 1
    else:
        b = 0
    return b
    
baroque_warmth(26)

0

In [21]:
# Difference (Baroque traits)
def baroque_difference(id_painting):
    """
    This function obtains the difference between the Baroque traits levels and diverse metrics.
   """
    a = test_5['darkness_level'][id_painting] - thresholds_explanations[0][0]
    b = test_5['no_paleness_level'][id_painting] - thresholds_explanations[0][1]
    c = test_5['contrast_level'][id_painting] - thresholds_explanations[0][2] 
    d = test_5['red_colours'][id_painting] - thresholds_explanations[0][3]
    a_5 = test_5['darkness_level'][id_painting] - thresholds_explanations_median[0][0]
    b_5 = test_5['no_paleness_level'][id_painting] - thresholds_explanations_median[0][1]
    c_5 = test_5['contrast_level'][id_painting] - thresholds_explanations_median[0][2] 
    d_5 = test_5['red_colours'][id_painting] - thresholds_explanations_median[0][3]
    a_5 = test_5['darkness_level'][id_painting] - thresholds_explanations_geometric_mean[0][0]
    b_5 = test_5['no_paleness_level'][id_painting] - thresholds_explanations_geometric_mean[0][1]
    c_5 = test_5['contrast_level'][id_painting] - thresholds_explanations_geometric_mean[0][2] 
    d_5 = test_5['red_colours'][id_painting] - thresholds_explanations_geometric_mean[0][3]
    return a+b+c+d+a_5+b_5+c_5+d_5+a_5+b_5+c_5+d_5
baroque_difference(1)

-1.5718698955304178

In [22]:
# Tie breaker function associated to the Baroque style. 
def baroque_tie_breaker(id_painting):
    """
    This function obtains the Baroque tie breaker metric.
   """
    a = 0.25*number_baroque_explanations(id_painting)
    b = 0.35*a
    c = 0.2*baroque_warmth(id_painting)
    d = 0.45*baroque_difference(id_painting)
    
    return b+c+d
#baroque_tie_breaker(19)

In [23]:
# Now we indicate which explanations would be activated
# if the Impressionism style is selected (testing dataset).

def number_impressionism_explanations(id_painting):
    """
    This function obtains the number of the "activated" explanations associated to the Impressionism style.
   """
    if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
        a = 1
    else: 
        if (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
            a = 1
        else: 
            if (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][7]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                a = 1
            else:
                if (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][7]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all(): 
                    a = 1
                else:
                    if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                        a = 2
                    else:
                        if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][7]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                            a = 2
                        else:
                            if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][5]).all(): 
                                a = 2
                            else:
                                if (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                                    a = 2
                                else:
                                    if (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all(): 
                                        a = 2
                                    else:
                                        if (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][5]).all(): 
                                            a = 2
                                        else:
                                            if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] < thresholds_explanations[0][7]).all(): 
                                                a = 3
                                            else:
                                                if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['bluish_level'][id_painting] < thresholds_explanations[0][6]).all(): 
                                                        a = 3
                                                else:
                                                    if (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofqcds'][id_painting] < thresholds_explanations[0][5]).all(): 
                                                        a = 3
                                                    else:
                                                        if (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofhues'][id_painting] < thresholds_explanations[0][4]).all(): 
                                                            a = 3
                                                        else:
                                                            if (test_5['diversityofqcds'][id_painting] >= thresholds_explanations[0][5]).all() and (test_5['bluish_level'][id_painting] >= thresholds_explanations[0][6]).all() and (test_5['greyish_level'][id_painting] >= thresholds_explanations[0][7]).all() and (test_5['diversityofhues'][id_painting] >= thresholds_explanations[0][4]).all(): 
                                                                a = 4
                                                            else:
                                                                a = 0
    return a



#number_impressionism_explanations(1)

In [24]:
# Warmth (Impressionism)
def impressionism_warmth(id_painting):
    """
    This function says if the level of warmth is higher than 0.32 and the red colors are less than 0.4.
   """
    if (test_5['warmth'][id_painting] >= 0.32 and test_5['red_colours'][id_painting] <= 0.4).all():
        b = 1
    else:
        b = 0
    return b
    
#impressionism_warmth(33)

In [25]:
# Difference (Impressionism traits)
def impressionism_difference(id_painting):
    """
    This function obtains the difference between the Impressionism traits levels and diverse metrics.
   """
    a = test_5['diversityofhues'][id_painting] - thresholds_explanations[0][4]
    b = test_5['diversityofqcds'][id_painting] - thresholds_explanations[0][5]
    c = test_5['bluish_level'][id_painting] - thresholds_explanations[0][6] 
    d = test_5['greyish_level'][id_painting] - thresholds_explanations[0][7]
    a_5 = test_5['diversityofhues'][id_painting] - thresholds_explanations_median[0][4]
    b_5 = test_5['diversityofqcds'][id_painting] - thresholds_explanations_median[0][5]
    c_5 = test_5['bluish_level'][id_painting] - thresholds_explanations_median[0][6] 
    d_5 = test_5['greyish_level'][id_painting] - thresholds_explanations_median[0][7]
    a_5 = test_5['diversityofhues'][id_painting] - thresholds_explanations_geometric_mean[0][4]
    b_5 = test_5['diversityofqcds'][id_painting] - thresholds_explanations_geometric_mean[0][5]
    c_5 = test_5['bluish_level'][id_painting] - thresholds_explanations_geometric_mean[0][6] 
    d_5 = test_5['greyish_level'][id_painting] - thresholds_explanations_geometric_mean[0][7]
    return a+b+c+d+a_5+b_5+c_5+d_5+a_5+b_5+c_5+d_5
#impressionism_difference(1)

In [26]:
# Tie breaker function associated to the impressionism style. 
def impressionism_tie_breaker(id_painting):
    """
    This function obtains the impressionism tie breaker metric.
   """
    a = 0.25*number_impressionism_explanations(id_painting)
    b = 0.35*a
    c = 0.2*impressionism_warmth(id_painting)
    d = 0.45*impressionism_difference(id_painting)
    
    return b+c+d
#impressionism_tie_breaker(29)

In [27]:
# Now we indicate which explanations would be activated
# if the Post-Impressionism style is selected (testing dataset).

def number_post_impressionism_explanations(id_painting):
    """
    This function obtains the number of the "activated" explanations associated to the post_impressionism style.
   """
    if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
        a = 1
    else: 
        if (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
            a = 1
        else: 
            if (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][11]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                a = 1
            else:
                if (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][11]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all(): 
                    a = 1
                else:
                    if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                        a = 2
                    else:
                        if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][11]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                            a = 2
                        else:
                            if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][9]).all(): 
                                a = 2
                            else:
                                if (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                                    a = 2
                                else:
                                    if (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all(): 
                                        a = 2
                                    else:
                                        if (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][9]).all(): 
                                            a = 2
                                        else:
                                            if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] < thresholds_explanations[0][11]).all(): 
                                                a = 3
                                            else:
                                                if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['contrast_yellow_blue'][id_painting] < thresholds_explanations[0][10]).all(): 
                                                        a = 3
                                                else:
                                                    if (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['warmth'][id_painting] < thresholds_explanations[0][9]).all(): 
                                                        a = 3
                                                    else:
                                                        if (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['vividness'][id_painting] < thresholds_explanations[0][8]).all(): 
                                                            a = 3
                                                        else:
                                                            if (test_5['warmth'][id_painting] >= thresholds_explanations[0][9]).all() and (test_5['contrast_yellow_blue'][id_painting] >= thresholds_explanations[0][10]).all() and (test_5['contrast_red_green'][id_painting] >= thresholds_explanations[0][11]).all() and (test_5['vividness'][id_painting] >= thresholds_explanations[0][8]).all(): 
                                                                a = 4
                                                            else:
                                                                a = 0
    return a



number_post_impressionism_explanations(13)

0

In [28]:
# Warmth (Post-Impressionism)
def post_impressionism_warmth(id_painting):
    """
    This function says if the level of warmth is higher than 0.32 and the vividness is higher than 0.5.
   """
    if (test_5['warmth'][id_painting] >= 0.32 and test_5['vividness'][id_painting] <= 0.5).all():
        b = 1
    else:
        b = 0
    return b
    
#post_impressionism_warmth(33)

In [29]:
# Difference (Post-Impressionism traits)
def post_impressionism_difference(id_painting):
    """
    This function obtains the difference between the Post-Impressionism traits levels and diverse metrics.
   """
    a = test_5['vividness'][id_painting] - thresholds_explanations[0][8]
    b = test_5['warmth'][id_painting] - thresholds_explanations[0][9]
    c = test_5['contrast_yellow_blue'][id_painting] - thresholds_explanations[0][10] 
    d = test_5['contrast_red_green'][id_painting] - thresholds_explanations[0][11]
    a_5 = test_5['vividness'][id_painting] - thresholds_explanations_median[0][8]
    b_5 = test_5['warmth'][id_painting] - thresholds_explanations_median[0][9]
    c_5 = test_5['contrast_yellow_blue'][id_painting] - thresholds_explanations_median[0][10] 
    d_5 = test_5['contrast_red_green'][id_painting] - thresholds_explanations_median[0][11]
    a_5 = test_5['vividness'][id_painting] - thresholds_explanations_geometric_mean[0][8]
    b_5 = test_5['warmth'][id_painting] - thresholds_explanations_geometric_mean[0][9]
    c_5 = test_5['contrast_yellow_blue'][id_painting] - thresholds_explanations_geometric_mean[0][10] 
    d_5 = test_5['contrast_red_green'][id_painting] - thresholds_explanations_geometric_mean[0][11]
    return a+b+c+d+a_5+b_5+c_5+d_5+a_5+b_5+c_5+d_5
#post_impressionism_difference(1)

In [30]:
# Tie breaker function associated to the Post-Impressionism style. 
def post_impressionism_tie_breaker(id_painting):
    """
    This function obtains the Post-Impressionism tie breaker metric.
   """
    a = 0.25*number_post_impressionism_explanations(id_painting)
    b = 0.35*a
    c = 0.2*post_impressionism_warmth(id_painting)
    d = 0.45*post_impressionism_difference(id_painting)
    
    return b+c+d
#post_impressionism_tie_breaker(29)

In [31]:
# To add the maximum column (partial definition of the dbAS function (Costa, Dellunde, Falomir, 2021, Logic Journal of the IGPL)).
test_5['dbas'] = test_5[['baroque_level', 'impressionism_level', 'postimpressionism_level']].max(axis=1)

In [32]:
# Obtaining the general accuracy (not balanced).
def baroqueaccuracy(dbas_painting11,dbas_painting12,dbas_painting13,dbas_painting14,
                            dbas_painting15,dbas_painting16,dbas_painting17,
                            dbas_painting18,dbas_painting19,dbas_painting20,
                            dbas_painting21):
    """
    This function is obtains the accuracy for the Baroque paintings.
   """
   
    if test_5['dbas'][11] == test_5['baroque_level'][11] and test_5['dbas'][11] != test_5['impressionism_level'][11] and test_5['dbas'][11] != test_5['postimpressionism_level'][11]: 
        k = 1
    else: 
        if test_5['dbas'][11] == test_5['baroque_level'][11] and test_5['dbas'][11] == test_5['impressionism_level'][11] and test_5['dbas'][11] != test_5['postimpressionism_level'][11]: 
            if baroque_tie_breaker(11)>impressionism_tie_breaker(11):
                k = 1
            if baroque_tie_breaker(11) == impressionism_tie_breaker(11):
                k = "cannot break the tie"
            else: k = 0
        if test_5['dbas'][11] == test_5['baroque_level'][11] and test_5['dbas'][11] != test_5['impressionism_level'][11] and test_5['dbas'][11] == test_5['postimpressionism_level'][11]: 
            if baroque_tie_breaker(11)>post_impressionism_tie_breaker(11):
                k = 1
            if baroque_tie_breaker(11) == post_impressionism_tie_breaker(11):
                k = "cannot break the tie"
            else: k = 0 
        if test_5['dbas'][11] == test_5['baroque_level'][11] and test_5['dbas'][11] == test_5['impressionism_level'][11] and test_5['dbas'][11] == test_5['postimpressionism_level'][11]: 
            if (baroque_tie_breaker(11)>impressionism_tie_breaker(11) and baroque_tie_breaker(11)>post_impressionism_tie_breaker(11)).all():
                k = 1
            if (baroque_tie_breaker(11) == impressionism_tie_breaker(11) and baroque_tie_breaker(11) == post_impressionism_tie_breaker(11)).all():
                k = "cannot break the tie"
            else: k = 0
        else: k = 0
    if test_5['dbas'][12] == test_5['baroque_level'][12] and test_5['dbas'][12] != test_5['impressionism_level'][12] and test_5['dbas'][12] != test_5['postimpressionism_level'][12]: 
        a = 1
    else: 
        if test_5['dbas'][12] == test_5['baroque_level'][12] and test_5['dbas'][12] == test_5['impressionism_level'][12] and test_5['dbas'][12] != test_5['postimpressionism_level'][12]: 
            if baroque_tie_breaker(12)>impressionism_tie_breaker(12):
                a = 1
            if baroque_tie_breaker(12) == impressionism_tie_breaker(12):
                a = "cannot break the tie"
            else: a = 0
        if test_5['dbas'][12] == test_5['baroque_level'][12] and test_5['dbas'][12] != test_5['impressionism_level'][12] and test_5['dbas'][12] == test_5['postimpressionism_level'][12]: 
            if baroque_tie_breaker(12)>post_impressionism_tie_breaker(12):
                a = 1
            if baroque_tie_breaker(12) == post_impressionism_tie_breaker(12):
                a = "cannot break the tie"
            else: a = 0 
        if test_5['dbas'][12] == test_5['baroque_level'][12] and test_5['dbas'][12] == test_5['impressionism_level'][12] and test_5['dbas'][12] == test_5['postimpressionism_level'][12]: 
            if (baroque_tie_breaker(12)>impressionism_tie_breaker(12) and baroque_tie_breaker(12)>post_impressionism_tie_breaker(12)).all():
                a = 1
            if (baroque_tie_breaker(12) == impressionism_tie_breaker(12) and baroque_tie_breaker(12) == post_impressionism_tie_breaker(12)).all():
                a = "cannot break the tie"
            else: a = 0
        else: a = 0
    if test_5['dbas'][13] == test_5['baroque_level'][13]: 
        b = 1
    else: 
        if test_5['dbas'][13] == test_5['baroque_level'][13] and test_5['dbas'][13] == test_5['impressionism_level'][13] and test_5['dbas'][13] != test_5['postimpressionism_level'][13]: 
            if baroque_tie_breaker(13)>impressionism_tie_breaker(13):
                b =1
            if baroque_tie_breaker(13) == impressionism_tie_breaker(13):
                b ="cannot break the tie"
            else: b =0
        if test_5['dbas'][13] == test_5['baroque_level'][13] and test_5['dbas'][13] != test_5['impressionism_level'][13] and test_5['dbas'][13] == test_5['postimpressionism_level'][13]: 
            if baroque_tie_breaker(13)>post_impressionism_tie_breaker(13):
                b =1
            if baroque_tie_breaker(13) == post_impressionism_tie_breaker(13):
                b ="cannot break the tie"
            else: b =0 
        if test_5['dbas'][13] == test_5['baroque_level'][13] and test_5['dbas'][13] == test_5['impressionism_level'][13] and test_5['dbas'][13] == test_5['postimpressionism_level'][13]: 
            if (baroque_tie_breaker(13)>impressionism_tie_breaker(13) and baroque_tie_breaker(13)>post_impressionism_tie_breaker(13)).all():
                b =1
            if (baroque_tie_breaker(13) == impressionism_tie_breaker(13) and baroque_tie_breaker(13) == post_impressionism_tie_breaker(13)).all():
                b ="cannot break the tie"   
            else: b = 0
        else: b = 0
    if test_5['dbas'][14] == test_5['baroque_level'][14]: 
        c = 1
    else:
        if test_5['dbas'][14] == test_5['baroque_level'][14] and test_5['dbas'][14] == test_5['impressionism_level'][14] and test_5['dbas'][14] != test_5['postimpressionism_level'][14]: 
            if baroque_tie_breaker(14)>impressionism_tie_breaker(14):
                c = 1
            if baroque_tie_breaker(14) == impressionism_tie_breaker(14):
                c = "cannot break the tie"
            else: c = 0
        if test_5['dbas'][14] == test_5['baroque_level'][14] and test_5['dbas'][14] != test_5['impressionism_level'][14] and test_5['dbas'][14] == test_5['postimpressionism_level'][14]: 
            if baroque_tie_breaker(14)>post_impressionism_tie_breaker(14):
                c = 1
            if baroque_tie_breaker(14) == post_impressionism_tie_breaker(14):
                c = "cannot break the tie"
            else: c = 0 
        if test_5['dbas'][14] == test_5['baroque_level'][14] and test_5['dbas'][14] == test_5['impressionism_level'][14] and test_5['dbas'][14] == test_5['postimpressionism_level'][14]: 
            if (baroque_tie_breaker(14)>impressionism_tie_breaker(14) and baroque_tie_breaker(14)>post_impressionism_tie_breaker(14)).all():
                c = 1
            if (baroque_tie_breaker(14) == impressionism_tie_breaker(14) and baroque_tie_breaker(14) == post_impressionism_tie_breaker(14)).all():
                c = "cannot break the tie"        
            else: c = 0
        else: c = 0
    if test_5['dbas'][15] == test_5['baroque_level'][15]: 
        d = 1
    else: 
        if test_5['dbas'][15] == test_5['baroque_level'][15] and test_5['dbas'][15] == test_5['impressionism_level'][15] and test_5['dbas'][15] != test_5['postimpressionism_level'][15]: 
            if baroque_tie_breaker(15)>impressionism_tie_breaker(15):
                d =  1
            if baroque_tie_breaker(15) == impressionism_tie_breaker(15):
                d =  "cannot break the tie"
            else: d =  0
        if test_5['dbas'][15] == test_5['baroque_level'][15] and test_5['dbas'][15] != test_5['impressionism_level'][15] and test_5['dbas'][15] == test_5['postimpressionism_level'][15]: 
            if baroque_tie_breaker(15)>post_impressionism_tie_breaker(15):
                d =  1
            if baroque_tie_breaker(15) == post_impressionism_tie_breaker(15):
                d =  "cannot break the tie"
            else: d =  0 
        if test_5['dbas'][15] == test_5['baroque_level'][15] and test_5['dbas'][15] == test_5['impressionism_level'][15] and test_5['dbas'][15] == test_5['postimpressionism_level'][15]: 
            if (baroque_tie_breaker(15)>impressionism_tie_breaker(15) and baroque_tie_breaker(15)>post_impressionism_tie_breaker(15)).all():
                d =  1
            if (baroque_tie_breaker(15) == impressionism_tie_breaker(15) and baroque_tie_breaker(15) == post_impressionism_tie_breaker(15)).all():
                d =  "cannot break the tie"  
            else: d = 0
        else: d = 0
    if test_5['dbas'][16] == test_5['baroque_level'][16]: 
        e = 1
    else: 
        if test_5['dbas'][16] == test_5['baroque_level'][16] and test_5['dbas'][16] == test_5['impressionism_level'][16] and test_5['dbas'][16] != test_5['postimpressionism_level'][16]: 
            if baroque_tie_breaker(16)>impressionism_tie_breaker(16):
                e =  1
            if baroque_tie_breaker(16) == impressionism_tie_breaker(16):
                e =  "cannot break the tie"
            else: e =  0
        if test_5['dbas'][16] == test_5['baroque_level'][16] and test_5['dbas'][16] != test_5['impressionism_level'][16] and test_5['dbas'][16] == test_5['postimpressionism_level'][16]: 
            if baroque_tie_breaker(16)>post_impressionism_tie_breaker(16):
                e =  1
            if baroque_tie_breaker(16) == post_impressionism_tie_breaker(16):
                e =  "cannot break the tie"
            else: e =  0 
        if test_5['dbas'][16] == test_5['baroque_level'][16] and test_5['dbas'][16] == test_5['impressionism_level'][16] and test_5['dbas'][16] == test_5['postimpressionism_level'][16]: 
            if (baroque_tie_breaker(16)>impressionism_tie_breaker(16) and baroque_tie_breaker(16)>post_impressionism_tie_breaker(16)).all():
                e =  1
            if (baroque_tie_breaker(16) == impressionism_tie_breaker(16) and baroque_tie_breaker(16) == post_impressionism_tie_breaker(16)).all():
                e =  "cannot break the tie"  
            else: e = 0
        else: e = 0
    if test_5['dbas'][17] == test_5['baroque_level'][17]: 
        f = 1
    else: 
        if test_5['dbas'][17] == test_5['baroque_level'][17] and test_5['dbas'][17] == test_5['impressionism_level'][17] and test_5['dbas'][17] != test_5['postimpressionism_level'][17]: 
            if baroque_tie_breaker(17)>impressionism_tie_breaker(17):
                f =  1
            if baroque_tie_breaker(17) == impressionism_tie_breaker(17):
                f =  "cannot break the tie"
            else: f =  0
        if test_5['dbas'][17] == test_5['baroque_level'][17] and test_5['dbas'][17] != test_5['impressionism_level'][17] and test_5['dbas'][17] == test_5['postimpressionism_level'][17]: 
            if baroque_tie_breaker(17)>post_impressionism_tie_breaker(17):
                f =  1
            if baroque_tie_breaker(17) == post_impressionism_tie_breaker(17):
                f =  "cannot break the tie"
            else: f =  0 
        if test_5['dbas'][17] == test_5['baroque_level'][17] and test_5['dbas'][17] == test_5['impressionism_level'][17] and test_5['dbas'][17] == test_5['postimpressionism_level'][17]: 
            if (baroque_tie_breaker(17)>impressionism_tie_breaker(17) and baroque_tie_breaker(17)>post_impressionism_tie_breaker(17)).all():
                f =  1
            if (baroque_tie_breaker(17) == impressionism_tie_breaker(17) and baroque_tie_breaker(17) == post_impressionism_tie_breaker(17)).all():
                f =  "cannot break the tie"  
            else: f = 0
        else: f = 0
    if test_5['dbas'][18] == test_5['baroque_level'][18]: 
        g = 1
    else: 
        if test_5['dbas'][18] == test_5['baroque_level'][18] and test_5['dbas'][18] == test_5['impressionism_level'][18] and test_5['dbas'][18] != test_5['postimpressionism_level'][18]: 
            if baroque_tie_breaker(18)>impressionism_tie_breaker(18):
                g =  1
            if baroque_tie_breaker(18) == impressionism_tie_breaker(18):
                g =  "cannot break the tie"
            else: g =  0
        if test_5['dbas'][18] == test_5['baroque_level'][18] and test_5['dbas'][18] != test_5['impressionism_level'][18] and test_5['dbas'][18] == test_5['postimpressionism_level'][18]: 
            if baroque_tie_breaker(18)>post_impressionism_tie_breaker(18):
                g =  1
            if baroque_tie_breaker(18) == post_impressionism_tie_breaker(18):
                g =  "cannot break the tie"
            else: g =  0 
        if test_5['dbas'][18] == test_5['baroque_level'][18] and test_5['dbas'][18] == test_5['impressionism_level'][18] and test_5['dbas'][18] == test_5['postimpressionism_level'][18]: 
            if (baroque_tie_breaker(18)>impressionism_tie_breaker(18) and baroque_tie_breaker(18)>post_impressionism_tie_breaker(18)).all():
                g =  1
            if (baroque_tie_breaker(18) == impressionism_tie_breaker(18) and baroque_tie_breaker(18) == post_impressionism_tie_breaker(18)).all():
                g =  "cannot break the tie"  
            else: g = 0
        else: g = 0
    if test_5['dbas'][19] == test_5['baroque_level'][19]: 
        h = 1 
    else: 
        if test_5['dbas'][19] == test_5['baroque_level'][19] and test_5['dbas'][19] == test_5['impressionism_level'][19] and test_5['dbas'][19] != test_5['postimpressionism_level'][19]: 
            if baroque_tie_breaker(19)>impressionism_tie_breaker(19):
                h =  1
            if baroque_tie_breaker(19) == impressionism_tie_breaker(19):
                h =  "cannot break the tie"
            else: h =  0
        if test_5['dbas'][19] == test_5['baroque_level'][19] and test_5['dbas'][19] != test_5['impressionism_level'][19] and test_5['dbas'][19] == test_5['postimpressionism_level'][19]: 
            if baroque_tie_breaker(19)>post_impressionism_tie_breaker(19):
                h =  1
            if baroque_tie_breaker(19) == post_impressionism_tie_breaker(19):
                h =  "cannot break the tie"
            else: h =  0 
        if test_5['dbas'][19] == test_5['baroque_level'][19] and test_5['dbas'][19] == test_5['impressionism_level'][19] and test_5['dbas'][19] == test_5['postimpressionism_level'][19]: 
            if (baroque_tie_breaker(19)>impressionism_tie_breaker(19) and baroque_tie_breaker(19)>post_impressionism_tie_breaker(19)).all():
                h =  1
            if (baroque_tie_breaker(19) == impressionism_tie_breaker(19) and baroque_tie_breaker(19) == post_impressionism_tie_breaker(19)).all():
                h =  "cannot break the tie"
            else: h = 0
        else: h = 0
    if test_5['dbas'][20] == test_5['baroque_level'][20]:
        i = 1 
    else: 
        if test_5['dbas'][20] == test_5['baroque_level'][20] and test_5['dbas'][20] == test_5['impressionism_level'][20] and test_5['dbas'][20] != test_5['postimpressionism_level'][20]: 
            if baroque_tie_breaker(20)>impressionism_tie_breaker(20):
                i =  1
            if baroque_tie_breaker(20) == impressionism_tie_breaker(20):
                i =  "cannot break the tie"
            else: i =  0
        if test_5['dbas'][20] == test_5['baroque_level'][20] and test_5['dbas'][20] != test_5['impressionism_level'][20] and test_5['dbas'][20] == test_5['postimpressionism_level'][20]: 
            if baroque_tie_breaker(20)>post_impressionism_tie_breaker(20):
                i =  1
            if baroque_tie_breaker(20) == post_impressionism_tie_breaker(20):
                i =  "cannot break the tie"
            else: i =  0 
        if test_5['dbas'][20] == test_5['baroque_level'][20] and test_5['dbas'][20] == test_5['impressionism_level'][20] and test_5['dbas'][20] == test_5['postimpressionism_level'][20]: 
            if (baroque_tie_breaker(20)>impressionism_tie_breaker(20) and baroque_tie_breaker(20)>post_impressionism_tie_breaker(20)).all():
                i =  1
            if (baroque_tie_breaker(20) == impressionism_tie_breaker(20) and baroque_tie_breaker(20) == post_impressionism_tie_breaker(20)).all():
                i =  "cannot break the tie" 
            else: i = 0
        else: i = 0
    if test_5['dbas'][21] == test_5['baroque_level'][21]: 
        j = 1
    else: 
        if test_5['dbas'][21] == test_5['baroque_level'][21] and test_5['dbas'][21] == test_5['impressionism_level'][21] and test_5['dbas'][21] != test_5['postimpressionism_level'][21]: 
            if baroque_tie_breaker(21)>impressionism_tie_breaker(21):
                j =  1
            if baroque_tie_breaker(21) == impressionism_tie_breaker(21):
                j =  "cannot break the tie"
            else: j =  0
        if test_5['dbas'][21] == test_5['baroque_level'][21] and test_5['dbas'][21] != test_5['impressionism_level'][21] and test_5['dbas'][21] == test_5['postimpressionism_level'][21]: 
            if baroque_tie_breaker(21)>post_impressionism_tie_breaker(21):
                j =  1
            if baroque_tie_breaker(21) == post_impressionism_tie_breaker(21):
                j =  "cannot break the tie"
            else: j =  0 
        if test_5['dbas'][21] == test_5['baroque_level'][21] and test_5['dbas'][21] == test_5['impressionism_level'][21] and test_5['dbas'][21] == test_5['postimpressionism_level'][21]: 
            if (baroque_tie_breaker(21)>impressionism_tie_breaker(21) and baroque_tie_breaker(21)>post_impressionism_tie_breaker(21)).all():
                j =  1
            if (baroque_tie_breaker(21) == impressionism_tie_breaker(21) and baroque_tie_breaker(21) == post_impressionism_tie_breaker(21)).all():
                j =  "cannot break the tie" 
            else: j = 0
        else: j = 0
    return k+a+b+c+d+e+f+g+h+i+j

def impressionismaccuracy(dbas_painting0,dbas_painting1,dbas_painting2,
                            dbas_painting3,dbas_painting4,dbas_painting5,
                            dbas_painting6,dbas_painting7,dbas_painting8,
                            dbas_painting9,dbas_painting10):
    """
    This function is obtains the accuracy for the Impressionist paintings.
    """
    if test_5['dbas'][0] == test_5['impressionism_level'][0]: 
        a = 1 
    else: 
        if test_5['dbas'][0] == test_5['baroque_level'][0] and test_5['dbas'][0] == test_5['impressionism_level'][0] and test_5['dbas'][0] != test_5['postimpressionism_level'][0]: 
            if baroque_tie_breaker(0)>impressionism_tie_breaker(0):
                a =  1
            if baroque_tie_breaker(0) == impressionism_tie_breaker(0):
                a =  "cannot break the tie"
            else: a =  0
        if test_5['dbas'][0] == test_5['baroque_level'][0] and test_5['dbas'][0] != test_5['impressionism_level'][0] and test_5['dbas'][0] == test_5['postimpressionism_level'][0]: 
            if baroque_tie_breaker(0)>post_impressionism_tie_breaker(0):
                a =  1
            if baroque_tie_breaker(0) == post_impressionism_tie_breaker(0):
                a =  "cannot break the tie"
            else: a =  0 
        if test_5['dbas'][0] == test_5['baroque_level'][0] and test_5['dbas'][0] == test_5['impressionism_level'][0] and test_5['dbas'][0] == test_5['postimpressionism_level'][0]: 
            if (baroque_tie_breaker(0)>impressionism_tie_breaker(0) and baroque_tie_breaker(0)>post_impressionism_tie_breaker(0)).all():
                a =  1
            if (baroque_tie_breaker(0) == impressionism_tie_breaker(0) and baroque_tie_breaker(0) == post_impressionism_tie_breaker(0)).all():
                a =  "cannot break the tie" 
            else: a = 0
        else: a = 0
    if test_5['dbas'][1] == test_5['impressionism_level'][1]:
        b = 1 
    else: 
        if test_5['dbas'][1] == test_5['baroque_level'][1] and test_5['dbas'][1] == test_5['impressionism_level'][1] and test_5['dbas'][1] != test_5['postimpressionism_level'][1]: 
            if baroque_tie_breaker(1)>impressionism_tie_breaker(1):
                b =  1
            if baroque_tie_breaker(1) == impressionism_tie_breaker(1):
                b =  "cannot break the tie"
            else: b =  0
        if test_5['dbas'][1] == test_5['baroque_level'][1] and test_5['dbas'][1] != test_5['impressionism_level'][1] and test_5['dbas'][1] == test_5['postimpressionism_level'][1]: 
            if baroque_tie_breaker(1)>post_impressionism_tie_breaker(1):
                b =  1
            if baroque_tie_breaker(1) == post_impressionism_tie_breaker(1):
                b =  "cannot break the tie"
            else: b =  0 
        if test_5['dbas'][1] == test_5['baroque_level'][1] and test_5['dbas'][1] == test_5['impressionism_level'][1] and test_5['dbas'][1] == test_5['postimpressionism_level'][1]: 
            if (baroque_tie_breaker(1)>impressionism_tie_breaker(1) and baroque_tie_breaker(1)>post_impressionism_tie_breaker(1)).all():
                b =  1
            if (baroque_tie_breaker(1) == impressionism_tie_breaker(1) and baroque_tie_breaker(1) == post_impressionism_tie_breaker(1)).all():
                b =  "cannot break the tie" 
            else: b = 0
        else: b = 0
    if test_5['dbas'][2] == test_5['impressionism_level'][2]: 
        c = 1
    else: 
        if test_5['dbas'][2] == test_5['baroque_level'][2] and test_5['dbas'][2] == test_5['impressionism_level'][2] and test_5['dbas'][2] != test_5['postimpressionism_level'][2]: 
            if baroque_tie_breaker(2)>impressionism_tie_breaker(2):
                c =  1
            if baroque_tie_breaker(2) == impressionism_tie_breaker(2):
                c =  "cannot break the tie"
            else: c =  0
        if test_5['dbas'][2] == test_5['baroque_level'][2] and test_5['dbas'][2] != test_5['impressionism_level'][2] and test_5['dbas'][2] == test_5['postimpressionism_level'][2]: 
            if baroque_tie_breaker(2)>post_impressionism_tie_breaker(2):
                c =  1
            if baroque_tie_breaker(2) == post_impressionism_tie_breaker(2):
                c =  "cannot break the tie"
            else: c =  0 
        if test_5['dbas'][2] == test_5['baroque_level'][2] and test_5['dbas'][2] == test_5['impressionism_level'][2] and test_5['dbas'][2] == test_5['postimpressionism_level'][2]: 
            if (baroque_tie_breaker(2)>impressionism_tie_breaker(2) and baroque_tie_breaker(2)>post_impressionism_tie_breaker(2)).all():
                c =  1
            if (baroque_tie_breaker(2) == impressionism_tie_breaker(2) and baroque_tie_breaker(2) == post_impressionism_tie_breaker(2)).all():
                c =  "cannot break the tie" 
            else: c = 0
        else: c = 0
    if test_5['dbas'][3] == test_5['impressionism_level'][3]: 
        d = 1 
    else: 
        if test_5['dbas'][3] == test_5['baroque_level'][3] and test_5['dbas'][3] == test_5['impressionism_level'][3] and test_5['dbas'][3] != test_5['postimpressionism_level'][3]: 
            if baroque_tie_breaker(3)>impressionism_tie_breaker(3):
                d =  1
            if baroque_tie_breaker(3) == impressionism_tie_breaker(3):
                d =  "cannot break the tie"
            else: d =  0
        if test_5['dbas'][3] == test_5['baroque_level'][3] and test_5['dbas'][3] != test_5['impressionism_level'][3] and test_5['dbas'][3] == test_5['postimpressionism_level'][3]: 
            if baroque_tie_breaker(3)>post_impressionism_tie_breaker(3):
                d =  1
            if baroque_tie_breaker(3) == post_impressionism_tie_breaker(3):
                d =  "cannot break the tie"
            else: d =  0 
        if test_5['dbas'][3] == test_5['baroque_level'][3] and test_5['dbas'][3] == test_5['impressionism_level'][3] and test_5['dbas'][3] == test_5['postimpressionism_level'][3]: 
            if (baroque_tie_breaker(3)>impressionism_tie_breaker(3) and baroque_tie_breaker(3)>post_impressionism_tie_breaker(3)).all():
                d =  1
            if (baroque_tie_breaker(3) == impressionism_tie_breaker(3) and baroque_tie_breaker(3) == post_impressionism_tie_breaker(3)).all():
                d =  "cannot break the tie" 
            else: d = 0
        else: d = 0
    if test_5['dbas'][4] == test_5['impressionism_level'][4]:
        e = 1 
    else: 
        if test_5['dbas'][4] == test_5['baroque_level'][4] and test_5['dbas'][4] == test_5['impressionism_level'][4] and test_5['dbas'][4] != test_5['postimpressionism_level'][4]: 
            if baroque_tie_breaker(4)>impressionism_tie_breaker(4):
                e =  1
            if baroque_tie_breaker(4) == impressionism_tie_breaker(4):
                e =  "cannot break the tie"
            else: e =  0
        if test_5['dbas'][4] == test_5['baroque_level'][4] and test_5['dbas'][4] != test_5['impressionism_level'][4] and test_5['dbas'][4] == test_5['postimpressionism_level'][4]: 
            if baroque_tie_breaker(4)>post_impressionism_tie_breaker(4):
                e =  1
            if baroque_tie_breaker(4) == post_impressionism_tie_breaker(4):
                e =  "cannot break the tie"
            else: e =  0 
        if test_5['dbas'][4] == test_5['baroque_level'][4] and test_5['dbas'][4] == test_5['impressionism_level'][4] and test_5['dbas'][4] == test_5['postimpressionism_level'][4]: 
            if (baroque_tie_breaker(4)>impressionism_tie_breaker(4) and baroque_tie_breaker(4)>post_impressionism_tie_breaker(4)).all():
                e =  1
            if (baroque_tie_breaker(4) == impressionism_tie_breaker(4) and baroque_tie_breaker(4) == post_impressionism_tie_breaker(4)).all():
                e =  "cannot break the tie" 
            else: e = 0
        else: e = 0
    if test_5['dbas'][5] == test_5['impressionism_level'][5]: 
        f = 1
    else: 
        if test_5['dbas'][5] == test_5['baroque_level'][5] and test_5['dbas'][5] == test_5['impressionism_level'][5] and test_5['dbas'][5] != test_5['postimpressionism_level'][5]: 
            if baroque_tie_breaker(5)>impressionism_tie_breaker(5):
                f =  1
            if baroque_tie_breaker(5) == impressionism_tie_breaker(5):
                f =  "cannot break the tie"
            else: f =  0
        if test_5['dbas'][5] == test_5['baroque_level'][5] and test_5['dbas'][5] != test_5['impressionism_level'][5] and test_5['dbas'][5] == test_5['postimpressionism_level'][5]: 
            if baroque_tie_breaker(5)>post_impressionism_tie_breaker(5):
                f =  1
            if baroque_tie_breaker(5) == post_impressionism_tie_breaker(5):
                f =  "cannot break the tie"
            else: f =  0 
        if test_5['dbas'][5] == test_5['baroque_level'][5] and test_5['dbas'][5] == test_5['impressionism_level'][5] and test_5['dbas'][5] == test_5['postimpressionism_level'][5]: 
            if (baroque_tie_breaker(5)>impressionism_tie_breaker(5) and baroque_tie_breaker(5)>post_impressionism_tie_breaker(5)).all():
                f =  1
            if (baroque_tie_breaker(5) == impressionism_tie_breaker(5) and baroque_tie_breaker(5) == post_impressionism_tie_breaker(5)).all():
                f =  "cannot break the tie" 
            else: f = 0
        else: f = 0
    if test_5['dbas'][6] == test_5['impressionism_level'][6]: 
        g = 1 
    else: 
        if test_5['dbas'][6] == test_5['baroque_level'][6] and test_5['dbas'][6] == test_5['impressionism_level'][6] and test_5['dbas'][6] != test_5['postimpressionism_level'][6]: 
            if baroque_tie_breaker(6)>impressionism_tie_breaker(6):
                g =  1
            if baroque_tie_breaker(6) == impressionism_tie_breaker(6):
                g =  "cannot break the tie"
            else: g =  0
        if test_5['dbas'][6] == test_5['baroque_level'][6] and test_5['dbas'][6] != test_5['impressionism_level'][6] and test_5['dbas'][6] == test_5['postimpressionism_level'][6]: 
            if baroque_tie_breaker(6)>post_impressionism_tie_breaker(6):
                g =  1
            if baroque_tie_breaker(6) == post_impressionism_tie_breaker(6):
                g =  "cannot break the tie"
            else: g =  0 
        if test_5['dbas'][6] == test_5['baroque_level'][6] and test_5['dbas'][6] == test_5['impressionism_level'][6] and test_5['dbas'][6] == test_5['postimpressionism_level'][6]: 
            if (baroque_tie_breaker(6)>impressionism_tie_breaker(6) and baroque_tie_breaker(6)>post_impressionism_tie_breaker(6)).all():
                g =  1
            if (baroque_tie_breaker(6) == impressionism_tie_breaker(6) and baroque_tie_breaker(6) == post_impressionism_tie_breaker(6)).all():
                g =  "cannot break the tie" 
            else: g = 0
        else: g = 0
    if test_5['dbas'][7] == test_5['impressionism_level'][7]:
        h = 1 
    else: 
        if test_5['dbas'][7] == test_5['baroque_level'][7] and test_5['dbas'][7] == test_5['impressionism_level'][7] and test_5['dbas'][7] != test_5['postimpressionism_level'][7]: 
            if baroque_tie_breaker(7)>impressionism_tie_breaker(7):
                h =  1
            if baroque_tie_breaker(7) == impressionism_tie_breaker(7):
                h =  "cannot break the tie"
            else: h =  0
        if test_5['dbas'][7] == test_5['baroque_level'][7] and test_5['dbas'][7] != test_5['impressionism_level'][7] and test_5['dbas'][7] == test_5['postimpressionism_level'][7]: 
            if baroque_tie_breaker(7)>post_impressionism_tie_breaker(7):
                h =  1
            if baroque_tie_breaker(7) == post_impressionism_tie_breaker(7):
                h =  "cannot break the tie"
            else: h =  0 
        if test_5['dbas'][7] == test_5['baroque_level'][7] and test_5['dbas'][7] == test_5['impressionism_level'][7] and test_5['dbas'][7] == test_5['postimpressionism_level'][7]: 
            if (baroque_tie_breaker(7)>impressionism_tie_breaker(7) and baroque_tie_breaker(7)>post_impressionism_tie_breaker(7)).all():
                h =  1
            if (baroque_tie_breaker(7) == impressionism_tie_breaker(7) and baroque_tie_breaker(7) == post_impressionism_tie_breaker(7)).all():
                h =  "cannot break the tie" 
            else: h = 0
        else: h = 0
    if test_5['dbas'][8] == test_5['impressionism_level'][8]: 
        i = 1
    else: 
        if test_5['dbas'][8] == test_5['baroque_level'][8] and test_5['dbas'][8] == test_5['impressionism_level'][8] and test_5['dbas'][8] != test_5['postimpressionism_level'][8]: 
            if baroque_tie_breaker(8)>impressionism_tie_breaker(8):
                i =  1
            if baroque_tie_breaker(8) == impressionism_tie_breaker(8):
                i =  "cannot break the tie"
            else: i =  0
        if test_5['dbas'][8] == test_5['baroque_level'][8] and test_5['dbas'][8] != test_5['impressionism_level'][8] and test_5['dbas'][8] == test_5['postimpressionism_level'][8]: 
            if baroque_tie_breaker(8)>post_impressionism_tie_breaker(8):
                i =  1
            if baroque_tie_breaker(8) == post_impressionism_tie_breaker(8):
                i =  "cannot break the tie"
            else: i =  0 
        if test_5['dbas'][8] == test_5['baroque_level'][8] and test_5['dbas'][8] == test_5['impressionism_level'][8] and test_5['dbas'][8] == test_5['postimpressionism_level'][8]: 
            if (baroque_tie_breaker(8)>impressionism_tie_breaker(8) and baroque_tie_breaker(8)>post_impressionism_tie_breaker(8)).all():
                i =  1
            if (baroque_tie_breaker(8) == impressionism_tie_breaker(8) and baroque_tie_breaker(8) == post_impressionism_tie_breaker(8)).all():
                i =  "cannot break the tie" 
            else: i = 0
        else: i = 0
    if test_5['dbas'][9] == test_5['impressionism_level'][9]: 
        j = 1
    else: 
        if test_5['dbas'][9] == test_5['baroque_level'][9] and test_5['dbas'][9] == test_5['impressionism_level'][9] and test_5['dbas'][9] != test_5['postimpressionism_level'][9]: 
            if baroque_tie_breaker(9)>impressionism_tie_breaker(9):
                j =  1
            if baroque_tie_breaker(9) == impressionism_tie_breaker(9):
                j =  "cannot break the tie"
            else: j =  0
        if test_5['dbas'][9] == test_5['baroque_level'][9] and test_5['dbas'][9] != test_5['impressionism_level'][9] and test_5['dbas'][9] == test_5['postimpressionism_level'][9]: 
            if baroque_tie_breaker(9)>post_impressionism_tie_breaker(9):
                j =  1
            if baroque_tie_breaker(9) == post_impressionism_tie_breaker(9):
                j =  "cannot break the tie"
            else: j =  0 
        if test_5['dbas'][9] == test_5['baroque_level'][9] and test_5['dbas'][9] == test_5['impressionism_level'][9] and test_5['dbas'][9] == test_5['postimpressionism_level'][9]: 
            if (baroque_tie_breaker(9)>impressionism_tie_breaker(9) and baroque_tie_breaker(9)>post_impressionism_tie_breaker(9)).all():
                j =  1
            if (baroque_tie_breaker(9) == impressionism_tie_breaker(9) and baroque_tie_breaker(9) == post_impressionism_tie_breaker(9)).all():
                j =  "cannot break the tie" 
            else: j = 0
        else: j = 0
    if test_5['dbas'][10] == test_5['impressionism_level'][10]: 
        k = 1
    else: 
        if test_5['dbas'][10] == test_5['baroque_level'][10] and test_5['dbas'][10] == test_5['impressionism_level'][10] and test_5['dbas'][10] != test_5['postimpressionism_level'][10]: 
            if baroque_tie_breaker(10)>impressionism_tie_breaker(10):
                k =  1
            if baroque_tie_breaker(10) == impressionism_tie_breaker(10):
                k =  "cannot break the tie"
            else: k =  0
        if test_5['dbas'][10] == test_5['baroque_level'][10] and test_5['dbas'][10] != test_5['impressionism_level'][10] and test_5['dbas'][10] == test_5['postimpressionism_level'][10]: 
            if baroque_tie_breaker(10)>post_impressionism_tie_breaker(10):
                k =  1
            if baroque_tie_breaker(10) == post_impressionism_tie_breaker(10):
                k =  "cannot break the tie"
            else: k =  0 
        if test_5['dbas'][10] == test_5['baroque_level'][10] and test_5['dbas'][10] == test_5['impressionism_level'][10] and test_5['dbas'][10] == test_5['postimpressionism_level'][10]: 
            if (baroque_tie_breaker(10)>impressionism_tie_breaker(10) and baroque_tie_breaker(10)>post_impressionism_tie_breaker(10)).all():
                k =  1
            if (baroque_tie_breaker(10) == impressionism_tie_breaker(10) and baroque_tie_breaker(10) == post_impressionism_tie_breaker(10)).all():
                k =  "cannot break the tie" 
            else: k = 0
        else: k = 0

    return a+b+c+d+e+f+g+h+i+j+k


def postimpressionismaccuracy(dbas_painting22,dbas_painting23,
                            dbas_painting24,dbas_painting25,dbas_painting26,
                           dbas_painting27,dbas_painting28,dbas_painting29,
                           dbas_painting30,dbas_painting31,dbas_painting32,
                           dbas_painting33):
    """
    This function is obtains the accuracy for the Post-Impressionits paintings.
    """
    if test_5['dbas'][22] == test_5['postimpressionism_level'][22]:
        j = 1 
    else: 
        if test_5['dbas'][22] == test_5['baroque_level'][22] and test_5['dbas'][22] == test_5['impressionism_level'][22] and test_5['dbas'][22] != test_5['postimpressionism_level'][22]: 
            if baroque_tie_breaker(22)>impressionism_tie_breaker(22):
                j =  1
            if baroque_tie_breaker(22) == impressionism_tie_breaker(22):
                j =  "cannot break the tie"
            else: j =  0
        if test_5['dbas'][22] == test_5['baroque_level'][22] and test_5['dbas'][22] != test_5['impressionism_level'][22] and test_5['dbas'][22] == test_5['postimpressionism_level'][22]: 
            if baroque_tie_breaker(22)>post_impressionism_tie_breaker(22):
                j =  1
            if baroque_tie_breaker(22) == post_impressionism_tie_breaker(22):
                j =  "cannot break the tie"
            else: j =  0 
        if test_5['dbas'][22] == test_5['baroque_level'][22] and test_5['dbas'][22] == test_5['impressionism_level'][22] and test_5['dbas'][22] == test_5['postimpressionism_level'][22]: 
            if (baroque_tie_breaker(22)>impressionism_tie_breaker(22) and baroque_tie_breaker(22)>post_impressionism_tie_breaker(22)).all():
                j =  1
            if (baroque_tie_breaker(22) == impressionism_tie_breaker(22) and baroque_tie_breaker(22) == post_impressionism_tie_breaker(22)).all():
                j =  "cannot break the tie" 
            else: j = 0
        else: j = 0
    if test_5['dbas'][23] == test_5['postimpressionism_level'][23]: 
        k = 1 
    else: 
        if test_5['dbas'][23] == test_5['baroque_level'][23] and test_5['dbas'][23] == test_5['impressionism_level'][23] and test_5['dbas'][23] != test_5['postimpressionism_level'][23]: 
            if baroque_tie_breaker(23)>impressionism_tie_breaker(23):
                k =  1
            if baroque_tie_breaker(23) == impressionism_tie_breaker(23):
                k =  "cannot break the tie"
            else: k =  0
        if test_5['dbas'][23] == test_5['baroque_level'][23] and test_5['dbas'][23] != test_5['impressionism_level'][23] and test_5['dbas'][23] == test_5['postimpressionism_level'][23]: 
            if baroque_tie_breaker(23)>post_impressionism_tie_breaker(23):
                k =  1
            if baroque_tie_breaker(23) == post_impressionism_tie_breaker(23):
                k =  "cannot break the tie"
            else: k =  0 
        if test_5['dbas'][23] == test_5['baroque_level'][23] and test_5['dbas'][23] == test_5['impressionism_level'][23] and test_5['dbas'][23] == test_5['postimpressionism_level'][23]: 
            if (baroque_tie_breaker(23)>impressionism_tie_breaker(23) and baroque_tie_breaker(23)>post_impressionism_tie_breaker(23)).all():
                k =  1
            if (baroque_tie_breaker(23) == impressionism_tie_breaker(23) and baroque_tie_breaker(23) == post_impressionism_tie_breaker(23)).all():
                k =  "cannot break the tie" 
            else: k = 0
        else: k = 0
    if test_5['dbas'][24] == test_5['postimpressionism_level'][24]: 
        l = 1 
    else: 
        if test_5['dbas'][24] == test_5['baroque_level'][24] and test_5['dbas'][24] == test_5['impressionism_level'][24] and test_5['dbas'][24] != test_5['postimpressionism_level'][24]: 
            if baroque_tie_breaker(24)>impressionism_tie_breaker(24):
                l =  1
            if baroque_tie_breaker(24) == impressionism_tie_breaker(24):
                l =  "cannot break the tie"
            else: l =  0
        if test_5['dbas'][24] == test_5['baroque_level'][24] and test_5['dbas'][24] != test_5['impressionism_level'][24] and test_5['dbas'][24] == test_5['postimpressionism_level'][24]: 
            if baroque_tie_breaker(24)>post_impressionism_tie_breaker(24):
                l =  1
            if baroque_tie_breaker(24) == post_impressionism_tie_breaker(24):
                l =  "cannot break the tie"
            else: l =  0 
        if test_5['dbas'][24] == test_5['baroque_level'][24] and test_5['dbas'][24] == test_5['impressionism_level'][24] and test_5['dbas'][24] == test_5['postimpressionism_level'][24]: 
            if (baroque_tie_breaker(24)>impressionism_tie_breaker(24) and baroque_tie_breaker(24)>post_impressionism_tie_breaker(24)).all():
                l =  1
            if (baroque_tie_breaker(24) == impressionism_tie_breaker(24) and baroque_tie_breaker(24) == post_impressionism_tie_breaker(24)).all():
                l =  "cannot break the tie" 
            else: l = 0
        else: l = 0
    if test_5['dbas'][25] == test_5['postimpressionism_level'][25]:
        m = 1 
    else: 
        if test_5['dbas'][25] == test_5['baroque_level'][25] and test_5['dbas'][25] == test_5['impressionism_level'][25] and test_5['dbas'][25] != test_5['postimpressionism_level'][25]: 
            if baroque_tie_breaker(25)>impressionism_tie_breaker(25):
                m =  1
            if baroque_tie_breaker(25) == impressionism_tie_breaker(25):
                m =  "cannot break the tie"
            else: m =  0
        if test_5['dbas'][25] == test_5['baroque_level'][25] and test_5['dbas'][25] != test_5['impressionism_level'][25] and test_5['dbas'][25] == test_5['postimpressionism_level'][25]: 
            if baroque_tie_breaker(25)>post_impressionism_tie_breaker(25):
                m =  1
            if baroque_tie_breaker(25) == post_impressionism_tie_breaker(25):
                m =  "cannot break the tie"
            else: m =  0 
        if test_5['dbas'][25] == test_5['baroque_level'][25] and test_5['dbas'][25] == test_5['impressionism_level'][25] and test_5['dbas'][25] == test_5['postimpressionism_level'][25]: 
            if (baroque_tie_breaker(25)>impressionism_tie_breaker(25) and baroque_tie_breaker(25)>post_impressionism_tie_breaker(25)).all():
                m =  1
            if (baroque_tie_breaker(25) == impressionism_tie_breaker(25) and baroque_tie_breaker(25) == post_impressionism_tie_breaker(25)).all():
                m =  "cannot break the tie" 
            else: m = 0
        else: m = 0
    if test_5['dbas'][26] == test_5['postimpressionism_level'][26]: 
        n = 1
    else: 
        if test_5['dbas'][26] == test_5['baroque_level'][26] and test_5['dbas'][26] == test_5['impressionism_level'][26] and test_5['dbas'][26] != test_5['postimpressionism_level'][26]: 
            if baroque_tie_breaker(26)>impressionism_tie_breaker(26):
                n =  1
            if baroque_tie_breaker(26) == impressionism_tie_breaker(26):
                n =  "cannot break the tie"
            else: n =  0
        if test_5['dbas'][26] == test_5['baroque_level'][26] and test_5['dbas'][26] != test_5['impressionism_level'][26] and test_5['dbas'][26] == test_5['postimpressionism_level'][26]: 
            if baroque_tie_breaker(26)>post_impressionism_tie_breaker(26):
                n =  1
            if baroque_tie_breaker(26) == post_impressionism_tie_breaker(26):
                n =  "cannot break the tie"
            else: n =  0 
        if test_5['dbas'][26] == test_5['baroque_level'][26] and test_5['dbas'][26] == test_5['impressionism_level'][26] and test_5['dbas'][26] == test_5['postimpressionism_level'][26]: 
            if (baroque_tie_breaker(26)>impressionism_tie_breaker(26) and baroque_tie_breaker(26)>post_impressionism_tie_breaker(26)).all():
                n =  1
            if (baroque_tie_breaker(26) == impressionism_tie_breaker(26) and baroque_tie_breaker(26) == post_impressionism_tie_breaker(26)).all():
                n =  "cannot break the tie" 
            else: n = 0
        else: n = 0
    if test_5['dbas'][27] == test_5['postimpressionism_level'][27]: 
        o = 1 
    else: 
        if test_5['dbas'][27] == test_5['baroque_level'][27] and test_5['dbas'][27] == test_5['impressionism_level'][27] and test_5['dbas'][27] != test_5['postimpressionism_level'][27]: 
            if baroque_tie_breaker(27)>impressionism_tie_breaker(27):
                o =  1
            if baroque_tie_breaker(27) == impressionism_tie_breaker(27):
                o =  "cannot break the tie"
            else: o =  0
        if test_5['dbas'][27] == test_5['baroque_level'][27] and test_5['dbas'][27] != test_5['impressionism_level'][27] and test_5['dbas'][27] == test_5['postimpressionism_level'][27]: 
            if baroque_tie_breaker(27)>post_impressionism_tie_breaker(27):
                o =  1
            if baroque_tie_breaker(27) == post_impressionism_tie_breaker(27):
                o =  "cannot break the tie"
            else: o =  0 
        if test_5['dbas'][27] == test_5['baroque_level'][27] and test_5['dbas'][27] == test_5['impressionism_level'][27] and test_5['dbas'][27] == test_5['postimpressionism_level'][27]: 
            if (baroque_tie_breaker(27)>impressionism_tie_breaker(27) and baroque_tie_breaker(27)>post_impressionism_tie_breaker(27)).all():
                o =  1
            if (baroque_tie_breaker(27) == impressionism_tie_breaker(27) and baroque_tie_breaker(27) == post_impressionism_tie_breaker(27)).all():
                o =  "cannot break the tie" 
            else: o = 0
        else: o = 0
    if test_5['dbas'][28] == test_5['postimpressionism_level'][28]:
        p = 1 
    else: 
        if test_5['dbas'][28] == test_5['baroque_level'][28] and test_5['dbas'][28] == test_5['impressionism_level'][28] and test_5['dbas'][28] != test_5['postimpressionism_level'][28]: 
            if baroque_tie_breaker(28)>impressionism_tie_breaker(28):
                p =  1
            if baroque_tie_breaker(28) == impressionism_tie_breaker(28):
                p =  "cannot break the tie"
            else: p =  0
        if test_5['dbas'][28] == test_5['baroque_level'][28] and test_5['dbas'][28] != test_5['impressionism_level'][28] and test_5['dbas'][28] == test_5['postimpressionism_level'][28]: 
            if baroque_tie_breaker(28)>post_impressionism_tie_breaker(28):
                p =  1
            if baroque_tie_breaker(28) == post_impressionism_tie_breaker(28):
                p =  "cannot break the tie"
            else: p =  0 
        if test_5['dbas'][28] == test_5['baroque_level'][28] and test_5['dbas'][28] == test_5['impressionism_level'][28] and test_5['dbas'][28] == test_5['postimpressionism_level'][28]: 
            if (baroque_tie_breaker(28)>impressionism_tie_breaker(28) and baroque_tie_breaker(28)>post_impressionism_tie_breaker(28)).all():
                p =  1
            if (baroque_tie_breaker(28) == impressionism_tie_breaker(28) and baroque_tie_breaker(28) == post_impressionism_tie_breaker(28)).all():
                p =  "cannot break the tie" 
            else: p = 0
        else: p = 0
    if test_5['dbas'][29] == test_5['postimpressionism_level'][29]: 
        q = 1
    else: 
        if test_5['dbas'][29] == test_5['baroque_level'][29] and test_5['dbas'][29] == test_5['impressionism_level'][29] and test_5['dbas'][29] != test_5['postimpressionism_level'][29]: 
            if baroque_tie_breaker(29)>impressionism_tie_breaker(29):
                q =  1
            if baroque_tie_breaker(29) == impressionism_tie_breaker(29):
                q =  "cannot break the tie"
            else: q =  0
        if test_5['dbas'][29] == test_5['baroque_level'][29] and test_5['dbas'][29] != test_5['impressionism_level'][29] and test_5['dbas'][29] == test_5['postimpressionism_level'][29]: 
            if baroque_tie_breaker(29)>post_impressionism_tie_breaker(29):
                q =  1
            if baroque_tie_breaker(29) == post_impressionism_tie_breaker(29):
                q =  "cannot break the tie"
            else: q =  0 
        if test_5['dbas'][29] == test_5['baroque_level'][29] and test_5['dbas'][29] == test_5['impressionism_level'][29] and test_5['dbas'][29] == test_5['postimpressionism_level'][29]: 
            if (baroque_tie_breaker(29)>impressionism_tie_breaker(29) and baroque_tie_breaker(29)>post_impressionism_tie_breaker(29)).all():
                q =  1
            if (baroque_tie_breaker(29) == impressionism_tie_breaker(29) and baroque_tie_breaker(29) == post_impressionism_tie_breaker(29)).all():
                q =  "cannot break the tie" 
            else: q = 0
        else: q = 0
    if test_5['dbas'][30] == test_5['postimpressionism_level'][30]:
        r = 1 
    else: 
        if test_5['dbas'][30] == test_5['baroque_level'][30] and test_5['dbas'][30] == test_5['impressionism_level'][30] and test_5['dbas'][30] != test_5['postimpressionism_level'][30]: 
            if baroque_tie_breaker(30)>impressionism_tie_breaker(30):
                r = 1
            if baroque_tie_breaker(30) == impressionism_tie_breaker(30):
                r = "cannot break the tie"
            else: q = 0
        if test_5['dbas'][30] == test_5['baroque_level'][30] and test_5['dbas'][30] != test_5['impressionism_level'][30] and test_5['dbas'][30] == test_5['postimpressionism_level'][30]: 
            if baroque_tie_breaker(30)>post_impressionism_tie_breaker(30):
                r = 1
            if baroque_tie_breaker(30) == post_impressionism_tie_breaker(30):
                r = "cannot break the tie"
            else: r = 0 
        if test_5['dbas'][30] == test_5['baroque_level'][30] and test_5['dbas'][30] == test_5['impressionism_level'][30] and test_5['dbas'][30] == test_5['postimpressionism_level'][30]: 
            if (baroque_tie_breaker(30)>impressionism_tie_breaker(30) and baroque_tie_breaker(30)>post_impressionism_tie_breaker(30)).all():
                r = 1
            if (baroque_tie_breaker(30) == impressionism_tie_breaker(30) and baroque_tie_breaker(30) == post_impressionism_tie_breaker(30)).all():
                r = "cannot break the tie" 
            else: r = 0
        else: r = 0
    if test_5['dbas'][31] == test_5['postimpressionism_level'][31]:
        s = 1 
    else: 
        if test_5['dbas'][31] == test_5['baroque_level'][31] and test_5['dbas'][31] == test_5['impressionism_level'][31] and test_5['dbas'][31] != test_5['postimpressionism_level'][31]: 
            if baroque_tie_breaker(31)>impressionism_tie_breaker(31):
                s = 1
            if baroque_tie_breaker(31) == impressionism_tie_breaker(31):
                s =  "cannot break the tie"
            else: s = 0
        if test_5['dbas'][31] == test_5['baroque_level'][31] and test_5['dbas'][31] != test_5['impressionism_level'][31] and test_5['dbas'][31] == test_5['postimpressionism_level'][31]: 
            if baroque_tie_breaker(31)>post_impressionism_tie_breaker(31):
                s =  1
            if baroque_tie_breaker(31) == post_impressionism_tie_breaker(31):
                s =  "cannot break the tie"
            else: s =  0 
        if test_5['dbas'][31] == test_5['baroque_level'][31] and test_5['dbas'][31] == test_5['impressionism_level'][31] and test_5['dbas'][31] == test_5['postimpressionism_level'][31]: 
            if (baroque_tie_breaker(31)>impressionism_tie_breaker(31) and baroque_tie_breaker(31)>post_impressionism_tie_breaker(31)).all():
                s = 1
            if (baroque_tie_breaker(31) == impressionism_tie_breaker(31) and baroque_tie_breaker(31) == post_impressionism_tie_breaker(31)).all():
                s =  "cannot break the tie" 
            else: s = 0
        else: s = 0
    if test_5['dbas'][32] == test_5['postimpressionism_level'][32]:
        t = 1 
    else: 
        if test_5['dbas'][32] == test_5['baroque_level'][32] and test_5['dbas'][32] == test_5['impressionism_level'][32] and test_5['dbas'][32] != test_5['postimpressionism_level'][32]: 
            if baroque_tie_breaker(32)>impressionism_tie_breaker(32):
                t =  1
            if baroque_tie_breaker(32) == impressionism_tie_breaker(32):
                t =  "cannot break the tie"
            else: t =  0
        if test_5['dbas'][32] == test_5['baroque_level'][32] and test_5['dbas'][32] != test_5['impressionism_level'][32] and test_5['dbas'][32] == test_5['postimpressionism_level'][32]: 
            if baroque_tie_breaker(32)>post_impressionism_tie_breaker(32):
                t =  1
            if baroque_tie_breaker(32) == post_impressionism_tie_breaker(32):
                t =  "cannot break the tie"
            else: t =  0 
        if test_5['dbas'][32] == test_5['baroque_level'][32] and test_5['dbas'][32] == test_5['impressionism_level'][32] and test_5['dbas'][32] == test_5['postimpressionism_level'][32]: 
            if (baroque_tie_breaker(32)>impressionism_tie_breaker(32) and baroque_tie_breaker(32)>post_impressionism_tie_breaker(32)).all():
                t =  1
            if (baroque_tie_breaker(32) == impressionism_tie_breaker(32) and baroque_tie_breaker(32) == post_impressionism_tie_breaker(32)).all():
                t =  "cannot break the tie" 
            else: t = 0
        else: t = 0
    if test_5['dbas'][33] == test_5['postimpressionism_level'][33]:
        u = 1 
    else: 
        if test_5['dbas'][33] == test_5['baroque_level'][33] and test_5['dbas'][33] == test_5['impressionism_level'][33] and test_5['dbas'][33] != test_5['postimpressionism_level'][33]: 
            if baroque_tie_breaker(33)>impressionism_tie_breaker(33):
                u =  1
            if baroque_tie_breaker(33) == impressionism_tie_breaker(33):
                u =  "cannot break the tie"
            else: u =  0
        if test_5['dbas'][33] == test_5['baroque_level'][33] and test_5['dbas'][33] != test_5['impressionism_level'][33] and test_5['dbas'][33] == test_5['postimpressionism_level'][33]: 
            if baroque_tie_breaker(33)>post_impressionism_tie_breaker(33):
                u =  1
            if baroque_tie_breaker(33) == post_impressionism_tie_breaker(33):
                u =  "cannot break the tie"
            else: u =  0 
        if test_5['dbas'][33] == test_5['baroque_level'][33] and test_5['dbas'][33] == test_5['impressionism_level'][33] and test_5['dbas'][33] == test_5['postimpressionism_level'][33]: 
            if (baroque_tie_breaker(33)>impressionism_tie_breaker(33) and baroque_tie_breaker(33)>post_impressionism_tie_breaker(33)).all():
                u =  1
            if (baroque_tie_breaker(33) == impressionism_tie_breaker(33) and baroque_tie_breaker(33) == post_impressionism_tie_breaker(33)).all():
                u =  "cannot break the tie" 
            else: u = 0
        else: u = 0                                  
    return j+k+l+m+n+o+p+q+r+s+t+u

def notbalancedbalancedgeneralaccuracy(dbas_painting0,dbas_painting1,dbas_painting2,
                            dbas_painting3,dbas_painting4,dbas_painting5,
                            dbas_painting6,dbas_painting7,dbas_painting8,
                            dbas_painting9,dbas_painting10,dbas_painting11,
                            dbas_painting12,dbas_painting13,dbas_painting14,
                            dbas_painting15,dbas_painting16,dbas_painting17,
                            dbas_painting18,dbas_painting19,dbas_painting20,
                            dbas_painting21,dbas_painting22,dbas_painting23,
                            dbas_painting24,dbas_painting25,dbas_painting26,
                           dbas_painting27,dbas_painting28,dbas_painting29,
                           dbas_painting30,dbas_painting31,dbas_painting32,
                           dbas_painting33):
    """
    This function obtains the general accuracy of the test.
    """
    return 100*(baroqueaccuracy(test_5['dbas'][11],test_5['dbas'][12],test_5['dbas'][13],test_5['dbas'][14],test_5['dbas'][15],
                            test_5['dbas'][16],test_5['dbas'][17],test_5['dbas'][18],test_5['dbas'][19],
                            test_5['dbas'][20],test_5['dbas'][21])
                            
            +impressionismaccuracy(test_5['dbas'][0],test_5['dbas'][1],test_5['dbas'][2],
                                   test_5['dbas'][3],test_5['dbas'][4],test_5['dbas'][5],
                                   test_5['dbas'][6],test_5['dbas'][7],test_5['dbas'][8],
                                  test_5['dbas'][9],test_5['dbas'][10])
            +postimpressionismaccuracy(test_5['dbas'][22],test_5['dbas'][23],test_5['dbas'][24],
                                       test_5['dbas'][25],test_5['dbas'][26],test_5['dbas'][27],
                                       test_5['dbas'][28],test_5['dbas'][29],test_5['dbas'][30],
                                      test_5['dbas'][31],test_5['dbas'][32],test_5['dbas'][33]))/34

In [33]:
notbalancedbalancedgeneralaccuracy(test_5['dbas'][11],test_5['dbas'][12],test_5['dbas'][13],test_5['dbas'][14],test_5['dbas'][15],
                            test_5['dbas'][16],test_5['dbas'][17],test_5['dbas'][18],test_5['dbas'][19],test_5['dbas'][20],test_5['dbas'][21],
                        test_5['dbas'][0],test_5['dbas'][1],test_5['dbas'][2],
                                   test_5['dbas'][3],test_5['dbas'][4],test_5['dbas'][5],
                                   test_5['dbas'][6],test_5['dbas'][7],test_5['dbas'][8],
                                  test_5['dbas'][9],test_5['dbas'][10],
                        test_5['dbas'][22],test_5['dbas'][23],test_5['dbas'][24],
                                       test_5['dbas'][25],test_5['dbas'][26],test_5['dbas'][27],
                                       test_5['dbas'][28],test_5['dbas'][29],test_5['dbas'][30],
                                      test_5['dbas'][31],test_5['dbas'][32],test_5['dbas'][33])

58.8235294117647

In [34]:
test_5

,darkness_level,no_paleness_level,contrast_level,red_colours,diversityofhues,diversityofqcds,bluish_level,greyish_level,vividness,warmth,contrast_yellow_blue,contrast_red_green,Classification,baroque_level,impressionism_level,postimpressionism_level,dbas
0,0.549710,0.844991,0.691013,0.743860,1.000000,1.000000,1.000000,0.940049,0.965625,0.828328,1.000000,0.847811,2.0,0.000000,0.940049,0.641764,0.940049
1,0.710898,0.841768,0.855424,0.796491,1.000000,1.000000,1.000000,1.000000,0.922252,0.546995,1.000000,0.614244,2.0,0.204581,1.000000,0.083491,1.000000
2,0.599285,0.799014,0.786565,1.000000,1.000000,1.000000,0.900000,0.869485,0.970922,1.000000,0.837247,1.000000,2.0,0.184864,0.769485,0.808169,0.808169
3,0.580000,0.942617,0.622668,0.883333,1.000000,1.000000,0.900000,0.773324,1.000000,0.840251,1.000000,0.833738,2.0,0.028618,0.673324,0.673989,0.673989
4,0.253358,0.746840,0.492812,1.000000,1.000000,1.000000,1.000000,1.000000,0.930243,0.529877,1.000000,0.604133,2.0,0.000000,1.000000,0.064253,1.000000
5,0.550000,0.899625,0.633890,0.813333,1.000000,1.000000,1.000000,0.833326,0.973006,0.868297,1.000000,0.952604,2.0,0.000000,0.833326,0.793907,0.833326
6,0.869920,0.966567,0.889646,1.000000,0.727273,0.810811,0.900000,0.726107,1.000000,0.974730,0.825753,1.000000,2.0,0.726133,0.164190,0.800483,0.800483
7,0.669400,0.987568,0.668126,1.000000,0.818182,0.891892,0.900000,0.761443,0.930148,0.787190,0.825753,0.858048,2.0,0.325093,0.371516,0.401138,0.401138
8,0.316827,0.631598,0.671522,0.933333,1.000000,1.000000,1.000000,1.000000,0.977719,0.710387,0.975753,0.781244,2.0,0.000000,1.000000,0.445103,1.000000
9,0.483134,0.626990,0.842438,1.000000,0.818182,0.918919,0.900000,0.651923,1.000000,1.000000,0.964633,1.000000,2.0,0.000000,0.289023,0.964633,0.964633
